<a href="https://colab.research.google.com/github/SEOUL-ABSS/SHIPSHIP/blob/main/SONAR4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 환경 설정 및 필수 라이브러리 임포트

프로젝트에 필요한 라이브러리를 설치/임포트하고, 전역 상수 및 기본 설정을 정의합니다.

In [19]:
# Install tensorflow
!pip install -q tensorflow tensorflow-hub soundfile librosa

# ==============================================================================
# 1. 환경 설정 및 필수 라이브러리 임포트
# ==============================================================================
print("1. 환경 설정 및 라이브러리 임포트 중...")

# 필요한 라이브러리 임포트
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import soundfile as sf # WAV 파일 처리용
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm # 폰트 관리자 임포트
import requests # MBARI 데이터 다운로드용
import subprocess # 오프라인 환경 패키지 다운로드용 (선택 사항)
# from tensorflow_addons.optimizers import RectifiedAdam # 예시: TFA 옵티마이저 사용 시

print("라이브러리 임포트 완료.")

# ==============================================================================
# Matplotlib 한글 폰트 설정 (오류 수정 및 안정화)
# ==============================================================================
print("\nMatplotlib 한글 폰트 설정 중...")

# Colab 환경에서 나눔고딕 폰트 설치 및 설정 시도
# 설치 오류 방지를 위해 출력을 숨깁니다.
!sudo apt-get update > /dev/null # Ensure apt cache is updated
!sudo apt-get install -y fonts-nanum > /dev/null
!sudo fc-cache -fv > /dev_null

# 폰트 관리자 캐시 재로드 및 폰트 설정
try:
    # 나눔고딕 폰트 파일 경로 (Colab에 일반적으로 설치되는 위치)
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    if os.path.exists(font_path):
        fm.fontManager.addfont(font_path)
        plt.rc('font', family='NanumGothic') # 나눔고딕으로 설정
        plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지
        print("Matplotlib 폰트 설정 완료: NanumGothic")
    else:
        print(f"경고: 폰트 파일 '{font_path}'를 찾을 수 없습니다. 기본 폰트를 사용합니다.")
except Exception as e:
    print(f"Matplotlib 폰트 설정 중 오류 발생: {e}. 기본 폰트를 사용합니다.")

print("Matplotlib 폰트 설정 완료.")

# ==============================================================================
# 전역 상수 정의
# ==============================================================================
print("\n전역 상수 정의 중...")
YAMNET_SAMPLE_RATE = 16000
YAMNET_EMBEDDING_DIM = 1024 # YAMNet embedding dimension

# VGGish embedding dimension (usually 128)
# Based on TF Hub documentation for vggish/1, the output is 128-dimensional.
VGGISH_EMBEDDING_DIM = 128

# PANNs embedding dimension (using YAMNet placeholder for now)
# If using a real PANNs model, this dimension would need to be updated based on the model's output.
PANNS_EMBEDDING_DIM = YAMNET_EMBEDDING_DIM # Placeholder dimension, assuming similar output shape to YAMNet placeholder

# Dataset Paths
DEEPSHIP_BASE_PATH = '/content/DeepShip'
# Directory where MBARI Noise Data will be stored.
# Assuming this directory will contain .wav files acting as 'noise'.
MBARI_NOISE_BASE_DIR = '/content/MBARI_noise_data'

# List of DeepShip ship classes
DEEPSHIP_CLASSES = ['Cargo', 'Passengership', 'Tanker', 'Tug']

# List of models to process for comparison
MODELS_TO_PROCESS = ['YAMNet', 'PANNs', 'VGGish'] # Use 'PANNs' as the key for the placeholder

print("전역 상수 정의 완료.")

import os

# Assuming DEEPSHIP_BASE_PATH and MBARI_NOISE_BASE_DIR are defined

print("\nDeepShip 데이터 디렉토리 내용 확인:")
if os.path.exists(DEEPSHIP_BASE_PATH):
    print(f"'{DEEPSHIP_BASE_PATH}' 내용:")
    # List contents of the DeepShip base directory, focusing on the expected class folders
    expected_deepship_subdirs = DEEPSHIP_CLASSES # Use the global constant
    found_content = False
    for item in os.listdir(DEEPSHIP_BASE_PATH):
        item_path = os.path.join(DEEPSHIP_BASE_PATH, item)
        if os.path.isdir(item_path):
            print(f"  {item}/")
            # If it's an expected class directory, list some of its contents
            if item in expected_deepship_subdirs:
                 try:
                     files_in_class_dir = os.listdir(item_path)
                     print(f"    ({len(files_in_class_dir)} items)")
                     for f in files_in_class_dir[:5]: # List up to 5 files
                         print(f"      {f}")
                     if len(files_in_class_dir) > 5:
                         print("      ...")
                     found_content = True
                 except Exception as e:
                      print(f"    오류: 디렉토리 내용 확인 중 오류 발생: {e}")
            else:
                 # List contents of unexpected subdirectories briefly
                 try:
                      sub_items = os.listdir(item_path)
                      print(f"    ({len(sub_items)} items)")
                      for f in sub_items[:3]: # List up to 3 items in other subdirs
                          print(f"      {f}")
                      if len(sub_items) > 3:
                          print("      ...")
                 except Exception as e:
                      print(f"    오류: 디렉토리 내용 확인 중 오류 발생: {e}")

        elif os.path.isfile(item_path):
            print(f"  {item}")
            found_content = True

    if not found_content:
        print("  (디렉토리가 비어 있습니다)")

else:
    print(f"경고: DeepShip Base Path '{DEEPSHIP_BASE_PATH}'를 찾을 수 없습니다.")


print("\nMBARI 노이즈 데이터 디렉토리 내용 확인:")
if os.path.exists(MBARI_NOISE_BASE_DIR):
    print(f"'{MBARI_NOISE_BASE_DIR}' 내용:")
    found_noise_content = False
    for root, dirs, files in os.walk(MBARI_NOISE_BASE_DIR):
        level = root.replace(MBARI_NOISE_BASE_DIR, '').count(os.sep)
        indent = '  ' * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = '  ' * (level + 1)
        if dirs:
             for d in dirs[:5]: # List up to 5 subdirectories
                  print(f'{subindent}{d}/')
             if len(dirs) > 5:
                  print(f'{subindent}...')

        if files:
             print(f'{subindent}파일들 ({len(files)}개):')
             for f in files[:5]: # List up to 5 files
                 print(f'{subindent}{f}')
                 if f.endswith('.wav'):
                      found_noise_content = True # Found at least one wav file
             if len(files) > 5:
                  print(f'{subindent}...')
        if not dirs and not files:
             print(f'{subindent}(비어 있음)')


    if not found_noise_content:
        print("\n경고: MBARI 노이즈 데이터 디렉토리에서 .wav 파일을 찾지 못했습니다.")

else:
    print(f"경고: MBARI Noise Base Directory '{MBARI_NOISE_BASE_DIR}'를 찾을 수 없습니다.")

print("\n데이터 디렉토리 내용 확인 완료.")

# Install boto3 for S3 access
!pip install -q boto3
print("boto3 설치 완료.")

1. 환경 설정 및 라이브러리 임포트 중...
라이브러리 임포트 완료.

Matplotlib 한글 폰트 설정 중...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Matplotlib 폰트 설정 완료: NanumGothic
Matplotlib 폰트 설정 완료.

전역 상수 정의 중...
전역 상수 정의 완료.

DeepShip 데이터 디렉토리 내용 확인:
'/content/DeepShip' 내용:
  Cargo/
    (13 items)
      38.wav
      15.wav
      69.wav
      cargo-metafile
      27.wav
      ...
  Tanker/
    (29 items)
      10.wav
      30.wav
      38.wav
      18.wav
      13.wav
      ...
  README.txt
  .git/
    (12 items)
      objects
      logs
      shallow
      ...
  Tug/
    (4 items)
      40.wav
      9.wav
      49.wav
      tug-metafile
  Passengership/
    (21 items)
      30.wav
      37.wav
      23.wav
      27.wav
      31.wav
      ...

MBARI 노이즈 데이터 디렉토리 내용 확인:
'/content/MBARI_noise_data' 내용:
MBARI_noise_data/
  파일들 (10개):
  MARS-20180105T000000Z-16kHz.wav
  MARS-20180

## 2. 데이터 확보: DeepShip 클론 및 노이즈 데이터 준비

DeepShip 데이터셋을 클론하고, MBARI 노이즈 데이터 디렉토리를 준비합니다. 이전에 다운로드된 MBARI 노이즈 샘플 파일이 있다면 해당 디렉토리로 이동시킵니다.

**주의**: 실제 MBARI Pacific Sound 16kHz 데이터셋은 직접 다운로드 또는 접근 설정이 필요할 수 있습니다. 아래 코드는 DeepShip 클론 및 샘플 노이즈 파일 처리를 위한 예시입니다.

In [20]:
# ==============================================================================
# 2. 데이터 확보: DeepShip 클론 및 노이즈 데이터 준비 (MBARI 다운로드 포함)
# ==============================================================================
import boto3 # Ensure boto3 is imported
from botocore import UNSIGNED # Ensure UNSIGNED config is imported
from botocore.client import Config # Ensure Config is imported
from pathlib import Path # Ensure Path is imported
import io # Ensure io is imported for potential in-memory reads (though we're downloading)
from six.moves.urllib.request import urlopen # Ensure urlopen is imported if still needed (less likely with direct S3 download)
import os # Ensure os is imported
import subprocess # Ensure subprocess is imported

print("\n2. 데이터 확보: DeepShip 클론 및 노이즈 데이터 준비 중...")

# Check if DeepShip is already cloned
if not os.path.exists(DEEPSHIP_BASE_PATH):
    print(f"DeepShip 데이터셋 클론 중: {DEEPSHIP_BASE_PATH}")
    # Clone the DeepShip repository
    # Use --depth 1 to clone only the latest commit, saving time and space
    try:
        subprocess.run(['git', 'clone', '--depth', '1', 'https://github.com/irfankamboh/DeepShip.git', DEEPSHIP_BASE_PATH], check=True, capture_output=True)
        print("DeepShip 데이터셋 클론 완료.")
    except subprocess.CalledProcessError as e:
        print(f"오류: DeepShip 데이터셋 클론 실패: {e.stderr.decode()}")
        print("수동으로 https://github.com/irfankamboh/DeepShip.git 를 클론하거나 다운로드하여")
        print(f"'{DEEPSHIP_BASE_PATH}' 경로에 위치시켜주세요.")
    except Exception as e:
         print(f"오류: DeepShip 데이터셋 클론 중 예기치 않은 오류 발생: {e}")
else:
    print(f"DeepShip 데이터셋이 이미 존재합니다: {DEEPSHIP_BASE_PATH}")

# Ensure the MBARI noise base directory exists
os.makedirs(MBARI_NOISE_BASE_DIR, exist_ok=True)
print(f"MBARI 노이즈 데이터 디렉토리 확인/생성 완료: {MBARI_NOISE_BASE_DIR}")

# --- Check if MBARI Noise Data already exists ---
# Count the number of .wav files in the MBARI_NOISE_BASE_DIR
existing_noise_files = [f for f in os.listdir(MBARI_NOISE_BASE_DIR) if f.endswith('.wav')]

if existing_noise_files:
    print(f"\nMBARI 노이즈 데이터가 지정된 디렉토리('{MBARI_NOISE_BASE_DIR}')에 이미 존재합니다. 다운로드를 건너뜁니다. (발견된 .wav 파일 수: {len(existing_noise_files)})")
else:
    # --- MBARI Noise Data Download ---
    # Use Boto3 to access the public S3 bucket and download a limited number of files
    # Based on the provided documentation example.
    s3_client = boto3.client('s3',
        aws_access_key_id='',
        aws_secret_access_key='',
        config=Config(signature_version=UNSIGNED))

    bucket = 'pacific-sound-16khz'
    # Define a prefix to narrow down the files (e.g., a specific year and month)
    # The documentation example uses '2018/01/'. Let's keep this or choose another if needed.
    prefix = '2018/01/' # Using January 2018 data as example

    # Limit the number of files to download to avoid excessive processing time and storage
    MAX_NOISE_FILES_TO_DOWNLOAD = 10 # Set the limit to 10 as requested

    print(f"\nMBARI 노이즈 데이터 다운로드 시도 중 (S3 버킷: {bucket}, Prefix: {prefix}, 최대 {MAX_NOISE_FILES_TO_DOWNLOAD} 파일):")

    try:
        # List objects in the specified bucket and prefix, potentially in pages
        paginator = s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket, Prefix=prefix)

        downloaded_count = 0
        found_any_objects = False # Track if any objects were found at all

        for page in pages:
            if 'Contents' in page:
                found_any_objects = True
                # print(f"  페이지에서 {len(page['Contents'])}개의 파일 발견. 다운로드 가능한 .wav 파일 탐색 중...") # Too verbose

                for obj in page['Contents']:
                    key = obj['Key']
                    # Only download .wav files and avoid directories or empty files
                    if key.endswith('.wav') and obj.get('Size', 0) > 0:
                        # Construct the local file path to save within the MBARI_NOISE_BASE_DIR
                        local_file_path = os.path.join(MBARI_NOISE_BASE_DIR, os.path.basename(key))

                        # Check if the file already exists locally to avoid re-downloading
                        if os.path.exists(local_file_path):
                            # print(f"    파일이 이미 존재합니다. 건너뜁니다: {os.path.basename(key)}") # Too verbose
                            pass # Skip if file already exists
                        else:
                            print(f"    다운로드 중: {os.path.basename(key)}...")
                            try:
                                s3_client.download_file(bucket, key, local_file_path)
                                downloaded_count += 1
                                print(f"      다운로드 완료 ({downloaded_count}/{MAX_NOISE_FILES_TO_DOWNLOAD})")
                            except Exception as download_e:
                                 print(f"    오류: 파일 다운로드 실패 ({os.path.basename(key)}): {download_e}")

                        # Stop downloading once the limit is reached
                        if downloaded_count >= MAX_NOISE_FILES_TO_DOWNLOAD:
                            print(f"\n  최대 다운로드 파일 수({MAX_NOISE_FILES_TO_DOWNLOAD})에 도달했습니다. 다운로드를 중지합니다.")
                            break # Break from the inner loop (files in this page)

                if downloaded_count >= MAX_NOISE_FILES_TO_DOWNLOAD:
                     break # Break from the outer loop (pages)


        if not found_any_objects:
            print(f"  경고: 지정된 Prefix '{prefix}'에서 파일을 찾을 수 없습니다.")
        elif downloaded_count == 0:
             # response might not be defined if no objects were found at all
             num_total_objects = 0
             try:
                  initial_response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix, MaxKeys=1) # Check if any object exists
                  if 'Contents' in initial_response:
                       num_total_objects = len(initial_response['Contents']) # This is just the first page count if MaxKeys > 1, or just 1 if MaxKeys=1
             except Exception:
                  pass # Ignore error if listing fails

             if num_total_objects > 0:
                  print(f"\n  지정된 Prefix '{prefix}'에 파일이 존재하지만, 다운로드 가능한 .wav 파일을 찾지 못했거나 모두 건너뛰었습니다.")
             else:
                  print(f"\n  지정된 Prefix '{prefix}'에 파일을 찾을 수 없습니다.")


        print(f"\n총 {downloaded_count}개의 노이즈 .wav 파일을 다운로드했습니다.")

    except Exception as e:
        print(f"오류: MBARI 노이즈 데이터 다운로드 중 오류 발생: {e}")
        print("S3 버킷 접근 권한, Prefix 설정, 또는 Boto3 설정/설치를 확인해주세요.")


# Note: To get sufficient noise data for meaningful training, you may need to adjust the 'prefix'
# or 'MAX_NOISE_FILES_TO_DOWNLOAD', or implement more complex logic to gather data from multiple
# prefixes/months, depending on your data needs and the S3 bucket structure.
# Ensure that the downloaded data includes enough samples from the 'noise' category.


print("\n2. 데이터 확보 단계 완료.")


2. 데이터 확보: DeepShip 클론 및 노이즈 데이터 준비 중...
DeepShip 데이터셋이 이미 존재합니다: /content/DeepShip
MBARI 노이즈 데이터 디렉토리 확인/생성 완료: /content/MBARI_noise_data

MBARI 노이즈 데이터가 지정된 디렉토리('/content/MBARI_noise_data')에 이미 존재합니다. 다운로드를 건너뜁니다. (발견된 .wav 파일 수: 10)

2. 데이터 확보 단계 완료.


## 3. 데이터 로드 및 준비 함수 정의

DeepShip 데이터와 노이즈 데이터를 수집하고, 'ship' 및 'noise' 레이블을 할당하며, 훈련 및 테스트 세트로 분할하는 함수를 정의합니다.

In [21]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming DEEPSHIP_BASE_PATH, MBARI_NOISE_BASE_DIR, DEEPSHIP_CLASSES are defined in previous cells.

def load_and_prepare_dataset(deepship_path, noise_data_dir, deepship_classes, test_size=0.2, random_state=42):
    """
    DeepShip 데이터셋과 노이즈 데이터를 로드하고 이진 분류(ship vs noise)를 위해 준비합니다.

    Args:
        deepship_path (str): DeepShip 데이터셋의 기본 경로.
        noise_data_dir (str): 노이즈 오디오 파일이 있는 디렉토리 경로.
        deepship_classes (list): DeepShip 데이터셋의 선박 클래스 이름 목록.
        test_size (float): 테스트 세트의 비율.
        random_state (int): 데이터 분할을 위한 랜덤 시드.

    Returns:
        tuple: (X_train_paths, X_test_paths, y_train_encoded, y_test_encoded, label_encoder, is_data_prepared, num_classes, noise_audio_paths)
               데이터 로드 및 준비 상태에 따라 결과가 달라질 수 있습니다.
               데이터 부족 시 X_train/test_paths, y_train/test_encoded는 빈 리스트/NumPy 배열이 됩니다.
               noise_audio_paths는 수집된 노이즈 파일 경로 목록입니다.
    """
    all_audio_paths = []
    all_labels = []
    noise_audio_paths = [] # List to store noise file paths for augmentation
    is_data_prepared = False
    label_encoder = None
    num_classes = 0

    print("\n데이터셋 로드 및 준비 시작: Ship vs Noise")
    print(f"DeepShip Base Path: {deepship_path}")
    print(f"MBARI Noise Data Directory: {noise_data_dir}")

    # --- 1. Integrate DeepShip Data ('ship') ---
    is_deepship_available = os.path.exists(deepship_path)
    if is_deepship_available:
        print(f"DeepShip 데이터셋에서 'ship' 오디오 파일 수집 중: {deepship_path}")
        found_ship_files = False
        # CORRECTED: Iterate directly through the class subdirectories at the top level of DeepShip
        for class_name in deepship_classes: # Use the provided deepship_classes list
            class_path = os.path.join(deepship_path, class_name)
            if os.path.isdir(class_path):
                print(f"  클래스 폴더 발견: {class_path} -> Class: {class_name}")
                for file_name in os.listdir(class_path):
                    if file_name.endswith('.wav'):
                        audio_path = os.path.join(class_path, file_name)
                        all_audio_paths.append(audio_path)
                        all_labels.append('ship') # Label all DeepShip ship types as 'ship'
                        found_ship_files = True
            else:
                 print(f"  경고: 예상 클래스 폴더 '{class_name}'를 '{deepship_path}'에서 찾을 수 없습니다.")


        if not found_ship_files:
            print(f"경고: '{deepship_path}' 내의 예상 클래스 폴더에서 'ship'으로 사용할 .wav 파일을 찾지 못했습니다. DeepShip 데이터셋 구조를 확인하세요.")
    else:
        print(f"경고: DeepShip 데이터셋을 찾을 수 없어 'ship' 데이터 수집을 건너뜁니다: {deepship_path}")


    # --- 2. Integrate Noise Data ('noise') ---
    is_noise_data_available_dir = os.path.exists(noise_data_dir)
    if is_noise_data_available_dir:
        print(f"노이즈 데이터 수집 중: {noise_data_dir}")
        found_noise_files = False
        # Collect all .wav files under the noise data directory
        for root, _, files in os.walk(noise_data_dir):
             # Exclude the DeepShip directory itself if it was downloaded into the noise dir by mistake
             if root.startswith(deepship_path):
                  continue
             for file_name in files:
                 if file_name.endswith('.wav'):
                     audio_path = os.path.join(root, file_name)
                     # Ensure we don't duplicate paths if DeepShip and downloaded DeepShip point to the same files
                     if audio_path not in all_audio_paths: # Avoid adding DeepShip files if they somehow ended up here
                         all_audio_paths.append(audio_path)
                         all_labels.append('noise') # 모든 노이즈 데이터를 'noise'로 레이블링
                         noise_audio_paths.append(audio_path) # Also add to noise_audio_paths for augmentation
                         found_noise_files = True

        if not found_noise_files:
            print(f"경고: '{noise_data_dir}'에서 'noise'로 사용할 .wav 파일을 찾지 못했습니다.")
    else:
        print(f"경고: 노이즈 데이터 디렉토리 '{noise_data_dir}'를 찾을 수 없어 'noise' 데이터 수집을 건너뜁니다.")
        print("실제 노이즈 데이터를 다운로드하여 이 디렉토리에 위치시켜주세요.")


    # --- 3. Data Preparation and Split ---
    unique_labels = np.unique(all_labels)

    # Check if data for both 'ship' and 'noise' classes is available and sufficient
    # We need at least 2 classes and some data for each class to perform a stratified split
    if len(all_audio_paths) > 0 and len(unique_labels) >= 2:
        # Check if each unique label has at least 2 samples for stratified split
        label_counts = pd.Series(all_labels).value_counts()
        if all(count >= 2 for count in label_counts):
            print(f"\n데이터 수집 완료. 총 샘플 수: {len(all_audio_paths)}")
            print(f"클래스 분포: {label_counts.to_dict()}")

            # 레이블 인코딩 ('ship', 'noise' 등 -> 0, 1 등)
            label_encoder = LabelEncoder()
            encoded_labels = label_encoder.fit_transform(all_labels)
            num_classes = len(label_encoder.classes_)
            print(f"레이블 인코딩 완료. 클래스: {label_encoder.classes_}, 총 {num_classes}개")

            # 데이터셋 분할 (훈련 및 테스트) - Stratified split으로 클래스 비율 유지
            X_train_paths, X_test_paths, y_train_encoded, y_test_encoded = train_test_split(
                all_audio_paths, encoded_labels, test_size=test_size, random_state=random_state, stratify=encoded_labels
            )

            print(f"데이터 분할 완료.")
            print(f"훈련 데이터 샘플 수: {len(X_train_paths)}")
            print(f"테스트 데이터 샘플 수: {len(X_test_paths)}")
            is_data_prepared = True # 데이터 준비 성공 플래그

        else:
             print("\n오류: 각 클래스별 샘플 수가 부족하여 데이터 분할(stratified split)을 수행할 수 없습니다.")
             print(f"클래스별 샘플 수: {label_counts.to_dict()}")
             is_data_prepared = False
             X_train_paths, X_test_paths, y_train_encoded, y_test_encoded = [], [], np.array([]), np.array([])


    else:
        print("\n오류: 'ship'과 'noise' 이진 분류를 위한 데이터가 충분하지 않습니다.")
        print(f"수집된 총 샘플 수: {len(all_audio_paths)}, 확인된 클래스: {unique_labels}")
        print("DeepShip 데이터셋이 올바르게 클론되었는지, 노이즈 데이터가 '{noise_data_dir}'에 충분히 있는지 확인해주세요.")
        is_data_prepared = False
        X_train_paths, X_test_paths, y_train_encoded, y_test_encoded = [], [], np.array([]), np.array([])


    print("\n데이터셋 로드 및 준비 함수 정의 완료.")

    # Return values regardless of success, check is_data_prepared flag later
    # Also return noise_audio_paths for potential augmentation
    return X_train_paths, X_test_paths, y_train_encoded, y_test_encoded, label_encoder, is_data_prepared, num_classes, noise_audio_paths

## 4. 오디오 전처리 및 임베딩 추출 함수 정의

각 오디오 모델(YAMNet, PANNs, VGGish)에 대한 오디오 전처리 및 임베딩 추출 함수를 정의합니다.

In [23]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import soundfile as sf
import librosa
import pandas as pd
import os
import time # 시간 측정을 위해 임포트
import sys # traceback 출력을 위해 임포트
import random # 노이즈 증강에 사용


# 오디오 파일을 로드하고 지정된 샘플링 속도로 리샘플링하는 함수 (Step 3에도 동일하게 정의됨)
# 이 함수는 이제 세그먼트 로딩 로직을 포함하도록 수정될 수 있습니다.
# 하지만 여기서는 단순 파일 로드 및 리샘플링 기능만 유지하고, 세그먼트 처리는 추출 함수 내부나 데이터 로드 단계에서 수행하도록 합니다.
# load_and_resample_audio 함수는 Step 3(셀 ID 5d281826)에서 정의된 최신 버전을 사용해야 합니다.
# 여기서는 함수 정의를 반복하지 않습니다. (동일 함수 중복 정의 방지)
# 만약 Step 3 셀 실행 전에 이 셀이 실행된다면 NameError가 발생할 수 있습니다.
# 따라서 Step 3 셀을 먼저 실행해야 합니다.

# Step 3 셀에서 정의된 load_and_resample_audio 함수가 사용 가능하도록 합니다.
# try-except 블록을 사용하여 함수 존재 여부 확인
try:
    load_and_resample_audio = load_and_resample_audio # Step 3에서 정의된 함수를 사용
    print("Step 3에서 load_and_resample_audio 함수를 성공적으로 가져왔습니다.")
except NameError:
    print("오류: Step 3 셀(load_and_prepare_dataset 함수 정의 셀)이 실행되지 않았거나 load_and_resample_audio 함수가 정의되지 않았습니다.")
    print("Step 3 셀을 먼저 실행해주세요.")
    # 함수가 없을 경우 더미 함수 정의 (오류 방지)
    def load_and_resample_audio(file_path, target_sample_rate):
        print(f"경고: load_and_resample_audio 함수가 정의되지 않아 파일 로드/리샘플링을 건너뜁니다: {file_path}")
        return None, None


# --- 1. YAMNet 임베딩 추출 함수 (세그먼트 처리 가능하도록 수정) ---
# YAMNet 모델 로드 (TensorFlow Hub 사용)
# 모델 로드는 한 번만 수행하도록 함수 외부에 정의
try:
    yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
    print("YAMNet 모델 로드 완료.")
except Exception as e:
    yamnet_model = None
    print(f"YAMNet 모델 로드 실패: {e}")
    print("YAMNet 임베딩 추출 기능을 사용할 수 없습니다.")


# YAMNet 임베딩 차원 정의 (모델 출력 차원에 맞춰야 함)
YAMNET_EMBEDDING_DIM = 1024 # YAMNet 모델의 임베딩 차원

# YAMNet 임베딩 추출 함수 수정: 파일 경로와 세그먼트 시작 시간을 인자로 받음
def extract_yamnet_embedding(audio_info: tuple, model, augment_with_noise: bool = False, noise_audio_paths: list = None, noise_level: float = 0.1, segment_duration_sec: int = 5, target_sample_rate: int = 16000):
    """
    오디오 세그먼트 정보(원본 파일 경로, 시작 시간)를 받아 YAMNet 임베딩을 추출합니다.
    필요 시 노이즈 증강을 적용합니다.

    Args:
        audio_info (tuple): (원본 파일 경로, 세그먼트 시작 시간(초)) 튜플.
        model: 로드된 YAMNet 모델 객체.
        augment_with_noise (bool): 노이즈 증강 적용 여부.
        noise_audio_paths (list): 노이즈 오디오 파일 경로 목록 (증강 시 사용).
        noise_level (float): 노이즈 레벨 (0.0 ~ 1.0).
        segment_duration_sec (int): 세그먼트 길이 (초).
        target_sample_rate (int): 임베딩 모델이 기대하는 샘플링 속도.

    Returns:
        np.ndarray: 추출된 임베딩 벡터. 임베딩 추출 실패 시 None 반환.
    """
    if model is None:
        # print("경고: YAMNet 모델이 로드되지 않았습니다. 임베딩 추출을 건너뜁니다.") # 너무 자세하면 생략
        return None

    file_path, start_time_sec = audio_info # 세그먼트 정보 언팩

    try:
        # 원본 오디오 파일 로드 및 리샘플링 (load_and_resample_audio 함수 사용)
        # 전체 파일을 로드하고, 필요한 세그먼트만 잘라냅니다.
        audio, sr = load_and_resample_audio(file_path, target_sample_rate)

        if audio is None or sr is None:
            # print(f"경고: '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 로드/리샘플링 실패.") # 너무 자세하면 생략
            return None

        # 세그먼트 시작 및 종료 샘플 인덱스 계산
        start_sample = int(start_time_sec * sr)
        end_sample = start_sample + int(segment_duration_sec * sr)

        # 세그먼트 오디오 데이터 추출
        # 오디오 길이가 세그먼트 종료 인덱스보다 짧을 경우, 가능한 만큼만 추출
        segment_audio = audio[start_sample:min(end_sample, len(audio))]

        # 세그먼트 길이가 임베딩 모델이 요구하는 최소 길이보다 짧을 경우 패딩 또는 건너뛰기
        # YAMNet은 약 0.96초 길이의 프레임을 처리하므로, 5초 세그먼트면 충분히 깁니다.
        # 하지만 마지막 세그먼트 길이가 짧을 수 있으므로 패딩 로직 추가 (필요 시)
        min_model_input_samples = int(0.96 * sr) # YAMNet 프레임 길이 (대략)
        if len(segment_audio) < min_model_input_samples:
            # print(f"경고: '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 길이가 짧습니다 ({len(segment_audio)} 샘플). 패딩 또는 건너뜁니다.")
            # 간단하게 패딩 (필요 시)
            # padded_segment = np.pad(segment_audio, (0, max(0, min_model_input_samples - len(segment_audio))), mode='constant')
            # segment_audio = padded_segment
            # 또는 건너뛰기
            return None # 길이가 너무 짧으면 건너뛰기


        # 노이즈 증강 적용 (훈련 데이터에만 적용)
        if augment_with_noise and noise_audio_paths and len(noise_audio_paths) > 0:
            try:
                 # 랜덤 노이즈 파일 선택
                 random_noise_file = random.choice(noise_audio_paths)
                 # 노이즈 오디오 로드 및 리샘플링
                 noise_audio, noise_sr = load_and_resample_audio(random_noise_file, target_sample_rate)

                 if noise_audio is not None and noise_sr is not None:
                      # 노이즈 세그먼트 추출 (랜덤 위치)
                      noise_segment_length = len(segment_audio)
                      if len(noise_audio) >= noise_segment_length:
                           noise_start_sample = random.randint(0, len(noise_audio) - noise_segment_length)
                           noise_segment = noise_audio[noise_start_sample:noise_start_sample + noise_segment_length]
                           # 원본 세그먼트와 노이즈 세그먼트 혼합
                           # 혼합 비율 계산 (예: noise_level에 따라)
                           alpha = 1.0 # 원본 오디오 비율
                           beta = noise_level # 노이즈 오디오 비율
                           # 두 오디오의 피크 레벨을 정규화하고 혼합 (간단한 혼합 예시)
                           mixed_segment = segment_audio + beta * noise_segment * (np.max(segment_audio) / (np.max(noise_segment) + 1e-8))
                           # 혼합된 오디오 클리핑 방지 및 정규화
                           mixed_segment = np.clip(mixed_segment, -1.0, 1.0)
                           segment_audio = mixed_segment # 증강된 세그먼트로 교체
                       else:
                           # print(f"경고: 노이즈 파일 '{os.path.basename(random_noise_file)}' 길이가 세그먼트 길이보다 짧습니다. 증강 건너뜁니다.") # 너무 자세하면 생략
                           pass # 노이즈 파일 길이가 짧으면 증강 건너뛰기
                 else:
                      # print(f"경고: 노이즈 파일 '{os.path.basename(random_noise_file)}' 로드 실패. 증강 건너뜁니다.") # 너무 자세하면 생략
                      pass # 노이즈 로드 실패 시 증강 건너뛰기

            except Exception as e:
                print(f"오류: 노이즈 증강 중 예외 발생 for '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초: {e}")
                import traceback
                traceback.print_exc(limit=2)
                # 오류 발생 시 증강 없이 원본 세그먼트로 진행


        # YAMNet 모델은 1초 길이의 오디오를 입력으로 받음. 5초 세그먼트에서 여러 프레임 추출 가능.
        # YAMNet 모델은 (num_samples,) 형태의 텐서를 기대.
        # 모델 추론을 위해 입력 형태 맞추기
        input_tensor = tf.constant(segment_audio, dtype=tf.float32)

        # YAMNet 모델 실행
        # model(input_tensor)는 (scores, embeddings, spectrogram) 튜플을 반환
        scores, embeddings, spectrogram = model(input_tensor)

        # YAMNet은 여러 프레임에 대한 임베딩을 반환하므로 평균 또는 최대 풀링하여 단일 벡터로 만듦
        # 여기서는 간단하게 시간 축(axis=0)에 대해 평균 풀링하여 단일 임베딩 벡터 생성
        if tf.shape(embeddings)[0] > 0:
             mean_embedding = tf.reduce_mean(embeddings, axis=0)
             return mean_embedding.numpy() # NumPy 배열로 변환하여 반환
        else:
             # print(f"경고: '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점에서 유효한 YAMNet 임베딩 추출 실패 (프레임 없음).") # 너무 자세하면 생략
             return None # 유효한 임베딩이 없으면 None 반환

    except Exception as e:
        print(f"오류: YAMNet 임베딩 추출 중 예외 발생 for '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초: {e}")
        import traceback
        traceback.print_exc(limit=2)
        return None


# --- 2. PANNs 임베딩 추출 함수 (세그먼트 처리 가능하도록 수정) ---
# PANNs 모델 로드는 복잡하므로, 로드 함수가 Step 5에 정의되어 있다고 가정합니다.
# 여기서는 모델 객체를 인자로 받습니다.

# PANNs 임베딩 차원 정의 (모델 출력 차원에 맞춰야 함)
PANNS_EMBEDDING_DIM = 2048 # PANNs 모델의 임베딩 차원 (예시)

# PANNs 임베딩 추출 함수 수정: 파일 경로와 세그먼트 시작 시간을 인자로 받음
def extract_panns_embedding(audio_info: tuple, model, augment_with_noise: bool = False, noise_audio_paths: list = None, noise_level: float = 0.1, segment_duration_sec: int = 5, target_sample_rate: int = 16000):
    """
    오디오 세그먼트 정보(원본 파일 경로, 시작 시간)를 받아 PANNs 임베딩을 추출합니다.
    필요 시 노이즈 증강을 적용합니다.

    Args:
        audio_info (tuple): (원본 파일 경로, 세그먼트 시작 시간(초)) 튜플.
        model: 로드된 PANNs 모델 객체.
        augment_with_noise (bool): 노이즈 증강 적용 여부.
        noise_audio_paths (list): 노이즈 오디오 파일 경로 목록 (증강 시 사용).
        noise_level (float): 노이즈 레벨 (0.0 ~ 1.0).
        segment_duration_sec (int): 세그먼트 길이 (초). PANNs 입력 길이에 맞추거나 조정 필요.
        target_sample_rate (int): 임베딩 모델이 기대하는 샘플링 속도.

    Returns:
        np.ndarray: 추출된 임베딩 벡터. 임베딩 추출 실패 시 None 반환.
    """
    if model is None:
        # print("경고: PANNs 모델이 로드되지 않았습니다. 임베딩 추출을 건너뜁니다.") # 너무 자세하면 생략
        return None

    file_path, start_time_sec = audio_info # 세그먼트 정보 언팩

    try:
        # 원본 오디오 파일 로드 및 리샘플링 (load_and_resample_audio 함수 사용)
        audio, sr = load_and_resample_audio(file_path, target_sample_rate)

        if audio is None or sr is None:
            # print(f"경고: '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 로드/리샘플링 실패.") # 너무 자세하면 생략
            return None

        # 세그먼트 시작 및 종료 샘플 인덱스 계산
        start_sample = int(start_time_sec * sr)
        end_sample = start_sample + int(segment_duration_sec * sr) # 세그먼트 길이

        # 세그먼트 오디오 데이터 추출
        segment_audio = audio[start_sample:min(end_sample, len(audio))]

        # PANNs 모델은 보통 고정된 길이의 입력을 기대합니다 (예: 10초).
        # 현재 세그먼트 길이(5초)는 PANNs 모델이 기대하는 길이와 다를 수 있습니다.
        # PANNs 모델의 입력 길이에 맞춰 패딩 또는 잘라내기 로직이 필요할 수 있습니다.
        # 여기서는 간단하게 5초 세그먼트를 사용하되, PANNs 모델의 입력 요구사항에 따라 수정이 필요합니다.
        # PANNs 입력 길이에 대한 정보가 필요합니다. (예: 10초 = 160000 샘플)
        panns_input_length_samples = int(10 * sr) # PANNs 예시 입력 길이 (10초)

        if len(segment_audio) < panns_input_length_samples:
             # PANNs 입력 길이보다 짧으면 패딩
             # print(f"경고: '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 길이가 PANNs 입력 길이보다 짧습니다 ({len(segment_audio)} 샘플). 패딩합니다.")
             padded_segment = np.pad(segment_audio, (0, max(0, panns_input_length_samples - len(segment_audio))), mode='constant')
             segment_audio = padded_segment
        elif len(segment_audio) > panns_input_length_samples:
             # PANNs 입력 길이보다 길면 잘라내기 (앞부분 사용)
             # print(f"경고: '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 길이가 깁니다 ({len(segment_audio)} 샘플). PANNs 입력 길이에 맞춰 잘라냅니다.")
             segment_audio = segment_audio[:panns_input_length_samples]


        # 노이즈 증강 적용 (훈련 데이터에만 적용) - YAMNet과 동일한 로직 사용 가능
        if augment_with_noise and noise_audio_paths and len(noise_audio_paths) > 0:
             try:
                  random_noise_file = random.choice(noise_audio_paths)
                  noise_audio, noise_sr = load_and_resample_audio(random_noise_file, target_sample_rate)

                  if noise_audio is not None and noise_sr is not None:
                       noise_segment_length = len(segment_audio) # 증강할 세그먼트 길이와 동일하게 노이즈 자름
                       if len(noise_audio) >= noise_segment_length:
                            noise_start_sample = random.randint(0, len(noise_audio) - noise_segment_length)
                            noise_segment = noise_audio[noise_start_sample:noise_start_sample + noise_segment_length]

                            alpha = 1.0
                            beta = noise_level
                            mixed_segment = segment_audio + beta * noise_segment * (np.max(segment_audio) / (np.max(noise_segment) + 1e-8))
                            mixed_segment = np.clip(mixed_segment, -1.0, 1.0)
                            segment_audio = mixed_segment
                       else:
                            pass # 노이즈 파일 길이 짧으면 증강 건너뛰기
                  else:
                       pass # 노이즈 로드 실패 시 증강 건너뛰기

             except Exception as e:
                 print(f"오류: PANNs 노이즈 증강 중 예외 발생 for '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초: {e}")
                 import traceback
                 traceback.print_exc(limit=2)
                 # 오류 발생 시 증강 없이 원본 세그먼트로 진행


        # PANNs 모델은 (batch_size, num_samples) 형태의 입력을 기대
        # 단일 세그먼트이므로 배치 차원 추가 ((num_samples,) -> (1, num_samples))
        input_tensor = tf.constant(segment_audio[np.newaxis, :], dtype=tf.float32) # 배치 차원 추가

        # PANNs 모델 실행 및 임베딩 추출
        # PANNs 모델의 출력 형태에 따라 임베딩 추출 로직 수정 필요
        # 예: model(input_tensor)['embedding'] 또는 model(input_tensor)[0] 등
        # PANNs 모델 객체의 정확한 사용법 확인 필요
        # 여기서는 예시로 'embedding' 키를 사용한다고 가정
        try:
            # PANNs 모델 추론 (모델 구조에 따라 호출 방식 다를 수 있음)
            # 예시: model(input_tensor)가 딕셔너리를 반환하고 'embedding' 키가 있다고 가정
            model_output = model(input_tensor)
            if isinstance(model_output, dict) and 'embedding' in model_output:
                 embedding = model_output['embedding']
                 # 결과는 (1, embedding_dim) 형태일 것이므로 배치 차원 제거
                 return embedding.numpy().squeeze(axis=0) # (embedding_dim,) 형태로 반환
            elif isinstance(model_output, (list, tuple)):
                 # 예시: model(input_tensor)가 (output, embedding) 튜플을 반환한다고 가정
                 if len(model_output) > 1:
                      embedding = model_output[1] # 두 번째 요소가 임베딩이라고 가정
                      return embedding.numpy().squeeze(axis=0)
                 else:
                      print(f"경고: PANNs 모델 출력 형태 예상과 다름 for '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초.")
                      return None
            else:
                print(f"경고: PANNs 모델 출력 형태 예상과 다름 for '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초.")
                return None


        except Exception as e:
            print(f"오류: PANNs 모델 추론 중 예외 발생 for '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초: {e}")
            import traceback
            traceback.print_exc(limit=2)
            return None

    except Exception as e:
        print(f"오류: PANNs 임베딩 추출 중 예외 발생 (로드/전처리 단계) for '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초: {e}")
        import traceback
        traceback.print_exc(limit=2)
        return None


# --- 3. VGGish 임베딩 추출 함수 (세그먼트 처리 가능하도록 수정) ---
# VGGish 모델 로드 (TensorFlow Hub 사용)
# 모델 로드는 한 번만 수행하도록 함수 외부에 정의
# try:
#     # VGGish 모델 및 관련 프리프로세싱 모델 로드
#     vggish_model = hub.load('https://tfhub.dev/google/vggish/1')
#     vggish_preprocess_model = hub.load('https://tfhub.dev/google/vggish-input/1')
#     print("VGGish 모델 및 프리프로세싱 모델 로드 완료.")
# except Exception as e:
#     vggish_model = None
#     vggish_preprocess_model = None
#     print(f"VGGish 모델 로드 실패: {e}")
#     print("VGGish 임베딩 추출 기능을 사용할 수 없습니다.")

# VGGish 임베딩 차원 정의
# VGGISH_EMBEDDING_DIM = 128 # VGGish 모델의 임베딩 차원

# VGGish 임베딩 추출 함수 수정: 파일 경로와 세그먼트 시작 시간을 인자로 받음
# def extract_vggish_embedding(audio_info: tuple, model, augment_with_noise: bool = False, noise_audio_paths: list = None, noise_level: float = 0.1, segment_duration_sec: int = 5, target_sample_rate: int = 16000):
#     """
#     오디오 세그먼트 정보(원본 파일 경로, 시작 시간)를 받아 VGGish 임베딩을 추출합니다.
#     필요 시 노이즈 증강을 적용합니다.

#     Args:
#         audio_info (tuple): (원본 파일 경로, 세그먼트 시작 시간(초)) 튜플.
#         model: 로드된 VGGish 모델 객체.
#         augment_with_noise (bool): 노이즈 증강 적용 여부.
#         noise_audio_paths (list): 노이즈 오디오 파일 경로 목록 (증강 시 사용).
#         noise_level (float): 노이즈 레벨 (0.0 ~ 1.0).
#         segment_duration_sec (int): 세그먼트 길이 (초). VGGish 입력 길이에 맞춰야 함.
#         target_sample_rate (int): 임베딩 모델이 기대하는 샘플링 속도.

#     Returns:
#         np.ndarray: 추출된 임베딩 벡터. 임베딩 추출 실패 시 None 반환.
#     """
#     if model is None or vggish_preprocess_model is None:
#         # print("경고: VGGish 모델 또는 프리프로세싱 모델이 로드되지 않았습니다. 임베딩 추출을 건너뜁니다.") # 너무 자세하면 생략
#         return None

#     file_path, start_time_sec = audio_info # 세그먼트 정보 언팩

#     try:
#         # 원본 오디오 파일 로드 및 리샘플링 (load_and_resample_audio 함수 사용)
#         audio, sr = load_and_resample_audio(file_path, target_sample_rate)

#         if audio is None or sr is None:
#             # print(f"경고: '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 로드/리샘플링 실패.") # 너무 자세하면 생략
#             return None

#         # 세그먼트 시작 및 종료 샘플 인덱스 계산
#         start_sample = int(start_time_sec * sr)
#         end_sample = start_sample + int(segment_duration_sec * sr) # 세그먼트 길이

#         # 세그먼트 오디오 데이터 추출
#         segment_audio = audio[start_sample:min(end_sample, len(audio))]

#         # VGGish 모델은 16kHz에서 0.96초 길이의 입력(15360 샘플)을 기대합니다.
#         # 현재 세그먼트 길이(5초)는 VGGish 입력 길이보다 깁니다.
#         # VGGish 프리프로세싱 모델은 긴 오디오를 받아서 0.96초 프레임으로 자동 분할하고 처리합니다.
#         # 따라서 5초 세그먼트를 그대로 프리프로세싱 모델에 전달합니다.

#         # 노이즈 증강 적용 (훈련 데이터에만 적용) - YAMNet과 동일한 로직 사용 가능
#         if augment_with_noise and noise_audio_paths and len(noise_audio_paths) > 0:
#              try:
#                   random_noise_file = random.choice(noise_audio_paths)
#                   noise_audio, noise_sr = load_and_resample_audio(random_noise_file, target_sample_rate)

#                   if noise_audio is not None and noise_sr is not None:
#                        noise_segment_length = len(segment_audio) # 증강할 세그먼트 길이와 동일하게 노이즈 자름
#                        if len(noise_audio) >= noise_segment_length:
#                             noise_start_sample = random.randint(0, len(noise_audio) - noise_segment_length)
#                             noise_segment = noise_audio[noise_start_sample:noise_start_sample + noise_segment_length]

#                             alpha = 1.0
#                             beta = noise_level
#                             mixed_segment = segment_audio + beta * noise_segment * (np.max(segment_audio) / (np.max(noise_segment) + 1e-8))
#                             mixed_segment = np.clip(mixed_segment, -1.0, 1.0)
#                             segment_audio = mixed_segment
#                        else:
#                             pass # 노이즈 파일 길이 짧으면 증강 건너뛰기
#                   else:
#                        pass # 노이즈 로드 실패 시 증강 건너뛰기

#              except Exception as e:
#                  print(f"오류: VGGish 노이즈 증강 중 예외 발생 for '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초: {e}")
#                  import traceback
#                  traceback.print_exc(limit=2)
#                  # 오류 발생 시 증강 없이 원본 세그먼트로 진행

#         # VGGish 프리프로세싱 모델 실행
#         # 입력은 (num_samples,) 형태의 텐서
#         input_tensor = tf.constant(segment_audio, dtype=tf.float32)
#         # VGGish 프리프로세싱 모델은 (num_frames, num_bands) 형태의 텐서를 반환
#         preprocess_output = vggish_preprocess_model(input_tensor)

#         # VGGish 모델 실행
#         # 입력은 (num_frames, num_bands) 형태의 텐서
#         # VGGish 모델은 (num_frames, embedding_dim) 형태의 임베딩을 반환
#         embeddings = model(preprocess_output)

#         # VGGish도 여러 프레임에 대한 임베딩을 반환하므로 평균 풀링하여 단일 벡터로 만듦
#         if tf.shape(embeddings)[0] > 0:
#              mean_embedding = tf.reduce_mean(embeddings, axis=0)
#              return mean_embedding.numpy() # NumPy 배열로 변환하여 반환
#         else:
#              # print(f"경고: '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점에서 유효한 VGGish 임베딩 추출 실패 (프레임 없음).") # 너무 자세하면 생략
#              return None # 유효한 임베딩이 없으면 None 반환

#     except Exception as e:
#         print(f"오류: VGGish 임베딩 추출 중 예외 발생 for '{os.path.basename(file_path)}' 세그먼트 시작 {start_time_sec:.2f}초: {e}")
#         import traceback
#         traceback.print_exc(limit=2)
#         return None


print("\n오디오 임베딩 추출 함수 정의 완료 (VGGish 관련 코드 삭제).")

# 이 셀 실행 후, Step 5 모델 로드 셀, 그리고 Step 6/10 파이프라인 실행 셀을 순서대로 실행해야 합니다.
# 특히 Step 3 셀(load_and_prepare_dataset 정의)이 먼저 실행되어 load_and_resample_audio 함수가 정의되어 있어야 합니다.

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 127)

## 5. 오디오 모델 로드 함수 정의

전이 학습에 사용할 YAMNet, PANNs, VGGish 사전 학습 모델을 TensorFlow Hub에서 로드하는 함수를 정의합니다.

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf # Ensure tensorflow is imported for model loading

print("\n5. 오디오 모델 로드 함수 정의 중...")

# Define the TensorFlow Hub handles for the models.
# Using the global constants defined in Step 1
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
vggish_model_handle = 'https://tfhub.dev/google/vggish/1'
# Using the YAMNet handle as a placeholder for PANNs as per previous discussion.
panns_model_handle = 'https://tfhub.dev/google/yamnet/1' # Using YAMNet as placeholder for PANNs

def load_audio_models():
    """
    TensorFlow Hub에서 YAMNet, PANNs, VGGish 모델을 로드합니다.

    Returns:
        dict: 로드된 모델 객체를 담고 있는 딕셔너리.
              모델 로드 실패 시 해당 모델 이름에 대해 값은 None이 됩니다.
    """
    models = {}
    are_models_loaded_successfully = True

    print("\n오디오 모델 로드 중...")

    # Load YAMNet
    try:
        print("  YAMNet 모델 로드 중...")
        models['YAMNet'] = hub.load(yamnet_model_handle)
        print("  YAMNet 모델 로드 완료.")
    except Exception as e:
        print(f"  오류: YAMNet 모델 로드 중 오류 발생: {e}")
        models['YAMNet'] = None # Set to None if loading fails
        are_models_loaded_successfully = False

    # Load PANNs (Placeholder)
    try:
        print("  PANNs 모델 로드 중 (YAMNet 플레이스홀더 사용)...")
        models['PANNs'] = hub.load(panns_model_handle)
        print("  PANNs 모델 로드 완료 (YAMNet 플레이스홀더).")
    except Exception as e:
        print(f"  오류: PANNs 모델 로드 중 오류 발생: {e}")
        models['PANNs'] = None # Set to None if loading fails
        are_models_loaded_successfully = False


    # Load VGGish
    try:
        print("  VGGish 모델 로드 중...")
        models['VGGish'] = hub.load(vggish_model_handle)
        print("  VGGish 모델 로드 완료.")
    except Exception as e:
        print(f"  오류: VGGish 모델 로드 중 오류 발생: {e}")
        models['VGGish'] = None # Set to None if loading fails
        are_models_loaded_successfully = False

    # Check if at least one model loaded successfully
    if all(model is None for model in models.values()):
        print("오류: 모든 오디오 모델 로드에 실패했습니다.")
        are_models_loaded_successfully = False # Ensure this is False if all failed
    else:
         print("모델 로드 상태:")
         for name, model in models.items():
             status = "Loaded" if model else "Failed"
             print(f"  {name}: {status}")


    print("\n오디오 모델 로드 함수 정의 완료.")

    # Return the dictionary of models. The caller must check for None values.
    return models, are_models_loaded_successfully

# Example usage (will be called in a later cell):
# loaded_models, are_models_loaded = load_audio_models()

## 6. 데이터 로드, 임베딩 추출 및 데이터 준비 실행

정의된 함수들을 사용하여 데이터셋을 로드하고, 각 모델별로 임베딩을 추출하며, 훈련 및 테스트를 위한 최종 데이터셋(임베딩 및 레이블)을 준비합니다.

In [ ]:
# Here is the current query...ㅇㅋ

# 필요한 라이브러리 임포트
import numpy as np
import tensorflow as tf
import pandas as pd
import os # 파일 시스템 경로 작업을 위한 라이브러리
import matplotlib.pyplot as plt # 데이터 시각화를 위한 라이브러리
import seaborn as sns # 데이터 시각화를 위한 라이브러리 (matplotlib 기반)
from sklearn.metrics import classification_report, confusion_matrix # 모델 평가 지표를 위한 라이브러리
import random # 데이터 증강에 사용될 수 있는 랜덤 모듈 (현재 코드에서는 직접 사용되지는 않음)
import tempfile # 임시 파일/디렉토리 생성을 위한 라이브러리
import shutil # 파일/디렉토리 복사, 삭제 등을 위한 라이브러리
import sys # 파이썬 런타임 정보 및 제어 (예: traceback 출력)
import gc # 가비지 컬렉터 인터페이스 (메모리 해제에 도움)
import soundfile as sf # 오디오 파일 로드/저장을 위한 라이브러리
import librosa # 오디오 분석 라이브러리 (리샘플링 등)

# 콜백 함수를 사용하기 위해 임포트 (모델 학습 단계에서 사용)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


print("\n10. 모델별 전체 파이프라인 실행 및 결과 비교 시작...")

# --- 중요: 필수 선행 작업 안내 ---
# 이 셀을 실행하기 전에 반드시 완료해야 하는 작업들을 안내합니다.
# 특히 Step 4 셀 실행과 충분한 데이터 확보가 중요합니다.
print("\n" + "="*80)
print(">>>         >>>  필수 작업 안내  <<<         <<<")
print(">>>                                           <<<")
print(">>>   오디오 전처리 및 임베딩 추출 함수가 정의된 Step 4 셀을 먼저 실행했는지 확인하세요! <<<")
print(">>>   (예: 셀 ID 617c87ca) 그렇지 않으면 'Quality must be one of...' 오류가 발생합니다. <<<")
print(">>>                                           <<<")
print(">>>   DeepShip('/content/DeepShip') 및 노이즈('/content/MBARI_noise_data') 디렉토리에 <<<")
print(">>>   훈련/테스트에 필요한 오디오 파일이 충분히 있는지 확인하세요!                     <<<")
print(">>>   (각 클래스 최소 2개 파일 필요) 데이터 부족 시 '데이터 부족' 오류가 발생합니다. <<<")
print(">>>                                           <<<")
print(">>>         >>>  필수 작업 완료 후 이 셀을 실행하세요.  <<<         <<<")
print("="*80 + "\n")


# --- 1. 데이터 로드 및 준비 ---
# Step 3에서 정의된 load_and_prepare_dataset 함수를 호출합니다.
# 이 함수가 정의되어 있는지 먼저 확인합니다.
# load_and_prepare_dataset 함수는 이제 세그먼트 정보(원본 파일 경로, 시작 시간)를 반환합니다.
if 'load_and_prepare_dataset' not in locals():
    print("오류: 'load_and_prepare_dataset' 함수가 정의되지 않았습니다. 파이프라인 실행을 중단합니다.")
    is_data_prepared = False # 데이터 준비 상태 플래그
    # 필요한 변수들을 빈 값으로 초기화합니다.
    # X_train_segment_info, X_test_segment_info는 이제 세그먼트 정보 리스트입니다.
    X_train_segment_info, X_test_segment_info, y_train_encoded, y_test_encoded, label_encoder, num_classes, noise_audio_paths = [], [], np.array([]), np.array([]), None, 0, [] # Initialize noise_audio_paths
else:
    # 함수가 정의되어 있으면 호출하여 데이터를 로드하고 나눕니다.
    # load_and_prepare_dataset 함수는 이제 세그먼트 정보 목록을 반환합니다.
    # (X_train_segment_info, X_test_segment_info, y_train_encoded, y_test_encoded, label_encoder, is_data_prepared, num_classes, noise_audio_paths)
    X_train_segment_info, X_test_segment_info, y_train_encoded, y_test_encoded, label_encoder, is_data_prepared, num_classes, noise_audio_paths = load_and_prepare_dataset(
        deepship_path=DEEPSHIP_BASE_PATH, # DeepShip 데이터 경로 (전역 변수 사용)
        noise_data_dir=MBARI_NOISE_BASE_DIR, # 노이즈 데이터 경로 (전역 변수 사용)
        deepship_classes=DEEPSHIP_CLASSES # 분류할 클래스 목록 (전역 변수 사용)
    )

# --- 2. 오디오 모델 로드 ---
# Step 5에서 정의된 load_audio_models 함수를 호출합니다.
# 이 함수가 정의되어 있는지 먼저 확인합니다. VGGish는 이 함수에서 제외되었습니다.
if 'load_audio_models' not in locals():
     print("오류: 'load_audio_models' 함수가 정의되지 않았습니다. 파이프라인 실행을 건너뜁니다.")
     are_models_loaded = False # 모델 로드 상태 플래그
     loaded_models = {} # 로드된 모델 저장 딕셔너리
else:
    # 함수가 정의되어 있으면 호출하여 오디오 임베딩 모델들(YAMNet, PANNs)을 로드합니다.
    loaded_models, are_models_loaded = load_audio_models()


# --- 3. Model-wise Embedding Extraction and Data Preparation (Batch Processing & Saving) ---
# 이 단계에서 각 오디오 세그먼트의 임베딩을 추출하고, 배치 단위로 처리하여 임시 파일에 저장합니다.
# 세그멘테이션 로직 적용으로 인해 이 부분의 루프 처리가 세그먼트 정보 목록을 사용하도록 변경됩니다.

temp_embedding_dir = None # 임시 임베딩 저장 디렉토리 경로
are_embeddings_extracted_successfully_any_model = False # 하나 이상의 모델에서 임베딩 추출 및 데이터 준비 성공 여부 플래그
embedding_dims = {} # 각 모델의 임베딩 차원을 저장할 딕셔너리

# 필요한 임베딩 추출 함수들(extract_yamnet_embedding, extract_panns_embedding)이
# Step 4 셀에서 정의되었는지 확인합니다. VGGish 함수는 제외되었습니다.
are_extraction_functions_defined = (
    'extract_yamnet_embedding' in locals() and
    'extract_panns_embedding' in locals() # VGGish 함수는 확인 대상에서 제외
)

# 임베딩 추출 함수 중 하나라도 정의되지 않았으면 이 단계를 건너뜠다는 메시지를 출력합니다.
if not are_extraction_functions_defined:
    print("오류: 필요한 임베딩 추출 함수 중 하나 이상이 정의되지 않았습니다 (YAMNet, PANNs). 임베딩 추출 단계를 건너뜁니다.")
    # 이 if 블록의 대응되는 else 블록(실제 임베딩 추출 로직 포함)을 건너뛰기 위해 pass 사용
    pass # 명시적인 pass 구문
# --- 여기서부터 SyntaxError가 보고되는 else 블록 시작 (이전 문제 지점) ---
# 임베딩 추출 함수들이 모두 정의되어 있을 때만 이 else 블록 내부 코드가 실행됩니다.
else: # 임베딩 추출 함수들이 정의된 경우
    # 임베딩 추출 및 데이터 준비의 메인 로직이 여기에 포함됩니다.

    # 데이터 로드 및 모델 로드가 성공적으로 완료되었는지 추가로 확인합니다.
    if is_data_prepared and are_models_loaded:
        print("\n모델별 임베딩 추출 및 데이터 준비 시작 (배치 처리 및 파일 저장 - 시간이 오래 걸릴 수 있습니다)...")

        # 임베딩을 저장할 임시 디렉토리를 생성합니다.
        temp_embedding_dir = tempfile.mkdtemp()
        print(f"임시 임베딩 저장 디렉토리: {temp_embedding_dir}")

        # 각 모델별 정보 (로드된 모델 객체, 임베딩 차원, 추출 함수)를 저장할 딕셔너리
        # VGGish 관련 정보는 여기서 제외합니다.
        models_info = {}
        # YAMNet 모델 정보 추가
        if loaded_models.get('YAMNet') is not None and 'YAMNET_EMBEDDING_DIM' in locals():
             models_info['YAMNet'] = {'model': loaded_models['YAMNet'], 'extract_func': extract_yamnet_embedding, 'dim': YAMNET_EMBEDDING_DIM}
        else:
             # YAMNet이 로드되지 않았거나 차원 정의 누락 시 경고
             print("경고: YAMNet 모델 정보 또는 함수가 누락되었습니다. YAMNet 처리를 건너뜁니다.")
        # PANNs 모델 정보 추가 (PANNs_EMBEDDING_DIM이 정의되어 있다고 가정)
        if loaded_models.get('PANNs') is not None and 'PANNS_EMBEDDING_DIM' in locals():
             models_info['PANNs'] = {'model': loaded_models['PANNs'], 'extract_func': extract_panns_embedding, 'dim': PANNS_EMBEDDING_DIM}
        else:
              # PANNs가 로드되지 않았거나 차원 정의 누락 시 경고
              print("경고: PANNs 모델 정보 또는 함수가 누락되었습니다. PANNs 처리를 건너뜁니다.")
        # VGGish 관련 코드는 삭제되었습니다.


        # 임베딩 추출을 위한 배치 크기를 정의합니다. 메모리 사용 최소화를 위해 1로 설정되었습니다.
        EMBEDDING_EXTRACTION_BATCH_SIZE = 1

        # 각 모델별로 저장된 배치 파일 목록과 필터링된 레이블을 저장할 딕셔너리
        # 이 딕셔너리들은 모델 학습/평가 단계에서 사용됩니다.
        X_train_embeddings = {} # 훈련 임베딩 배치 파일 경로 목록 (모델별)
        X_test_embeddings = {} # 테스트 임베딩 배치 파일 경로 목록 (모델별)
        y_train_filtered = {} # 훈련 필터링 레이블 (np.array, 모델별)
        y_test_filtered = {}  # 테스트 필터링 레이블 (np.array, 모델별)
        y_test_encoded_filtered = {} # 테스트 필터링 레이블 (평가 보고서용, 모델별)


        # 처리할 모델 정보가 하나라도 있는지 확인
        if models_info:
            all_models_extracted_at_least_one_sample = False # 하나 이상의 모델이 훈련/테스트 가능한 데이터를 추출했는지 추적하는 플래그

            # models_info에 있는 각 모델에 대해 임베딩 추출 및 데이터 준비 수행
            for model_name, info in models_info.items():
                model = info['model'] # 현재 모델 객체
                extract_func = info['extract_func'] # 현재 모델의 임베딩 추출 함수
                # 임베딩 차원 정보가 models_info에 있는지 확인 후 추가 (안전 장치)
                if 'dim' in info:
                    embedding_dims[model_name] = info['dim'] # 현재 모델의 임베딩 차원 저장
                else:
                    print(f"경고: {model_name}의 임베딩 차원 정보가 models_info에 없습니다. 임베딩 차원 딕셔너리에 추가하지 않습니다.")
                    # 임베딩 차원 없이는 모델 구축이 불가능하므로, 이 모델은 이후 단계에서 건너뛸 수 있습니다.


                print(f"\n--- {model_name} 임베딩 추출 중 (배치 크기: {EMBEDDING_EXTRACTION_BATCH_SIZE}) ---")
                train_batch_files = [] # 현재 모델의 훈련 임베딩 배치 파일 경로 목록
                test_batch_files = []  # 현재 모델의 테스트 임베딩 배치 파일 경로 목록

                # 훈련 데이터 세그먼트 정보 목록을 배치 처리
                print("  훈련 데이터 세그먼트 처리 중...")
                train_embeddings_batch = [] # 현재 훈련 배치에 포함될 임베딩 목록
                train_labels_batch = []     # 현재 훈련 배치에 포함될 레이블 목록

                # Step 1에서 반환된 훈련 세그먼트 정보 목록(X_train_segment_info) 사용
                # X_train_segment_info는 [(원본 파일 경로, 시작 시간), ...] 형태의 리스트
                # y_train_encoded는 세그먼트 수와 일치하는 인코딩된 레이블 배열
                for i, segment_info_tuple in enumerate(X_train_segment_info):
                    # 세그먼트 정보 튜플에서 원본 파일 경로와 시작 시간 언팩
                    original_file_path, start_time_sec = segment_info_tuple
                    # 해당 세그먼트의 레이블은 y_train_encoded 배열에서 인덱스로 가져옴
                    original_label_encoded = y_train_encoded[i]
                    # 레이블 인코더가 유효한 경우 레이블 문자열 가져오기
                    original_label_str = label_encoder.inverse_transform([original_label_encoded])[0] if label_encoder is not None and hasattr(label_encoder, 'inverse_transform') else str(original_label_encoded)

                    # 현재 처리 중인 세그먼트 정보 출력
                    # print(f"    처리 세그먼트 {i+1}/{len(X_train_segment_info)}: 파일 '{os.path.basename(original_file_path)}' 시작 {start_time_sec:.2f}초")
                    embedding = None # 임베딩 초기화 (오류 확인용)

                    # --- 임베딩 추출 시도 ---
                    try:
                         # extract_func 함수 호출하여 임베딩 추출
                         # extract_func는 세그먼트 정보 튜플을 인자로 받도록 수정되었습니다.
                         # 노이즈 증강은 'ship' 클래스 세그먼트에만 적용
                         augment = (original_label_str == 'ship') # 'ship' 클래스일 때만 증강 플래그 True
                         embedding = extract_func(
                              segment_info_tuple, # 세그먼트 정보 튜플 전달
                              model, # 현재 모델 객체
                              augment_with_noise=augment, # 증강 적용 여부
                              noise_audio_paths=noise_audio_paths, # 노이즈 파일 경로 목록
                              noise_level=0.1 # 노이즈 레벨
                              # segment_duration_sec, target_sample_rate 인자는 extract_func 내부에서 상수로 사용될 수 있음
                         )

                         # 임베딩 추출 성공 시 배치 목록에 추가
                         if embedding is not None:
                              train_embeddings_batch.append(embedding)
                              train_labels_batch.append(original_label_encoded)
                         else:
                              # 임베딩 추출 실패 시 경고 메시지 출력
                              print(f"      경고: 파일 '{os.path.basename(original_file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 임베딩 추출 실패 (결과 None).")

                    # 임베딩 추출 중 예외 발생 시 오류 메시지 출력 및 다음 세그먼트로 이동
                    except Exception as e:
                         print(f"    오류: 파일 '{os.path.basename(original_file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 임베딩 추출 중 예외 발생: {e}")
                         import traceback
                         traceback.print_exc(limit=2)
                         # 오류가 발생해도 루프를 중단하지 않고 다음 세그먼트로 계속 진행


                    # 배치 사이즈에 도달했거나 마지막 세그먼트인 경우 현재 배치 저장
                    if (len(train_embeddings_batch) >= EMBEDDING_EXTRACTION_BATCH_SIZE) or (i == len(X_train_segment_info) - 1 and train_embeddings_batch):
                        print(f"    저장 중: 訓練 배치 {len(train_batch_files) + 1}, 현재 배치 세그먼트 수: {len(train_embeddings_batch)}")
                        try:
                             # 배치 목록을 numpy 배열로 변환
                             batch_embeddings_array = np.array(train_embeddings_batch)
                             batch_labels_array = np.array(train_labels_batch)

                             # 배치 데이터를 임시 파일에 .npz 형식으로 저장
                             batch_file_path = os.path.join(temp_embedding_dir, f'{model_name}_train_batch_{len(train_batch_files)}.npz')
                             np.savez(batch_file_path, embeddings=batch_embeddings_array, labels=batch_labels_array)
                             train_batch_files.append(batch_file_path) # 저장된 파일 경로 목록에 추가

                             # 현재 배치 목록을 비우고 메모리 해제 시도
                             train_embeddings_batch = []
                             train_labels_batch = []
                             print(f"    訓練 배치 {len(train_batch_files)} 저장 완료. 메모리 해제.")
                             gc.collect() # 가비지 컬렉션 강제 실행


                        # 배치 저장 중 오류 발생 시 처리
                        except Exception as e:
                            print(f"    오류: 訓練 배치 저장 실패 - 배치 {len(train_batch_files) + 1}, 오류: {e}")
                            # 오류 발생 시에도 메모리 확보를 위해 배치 목록 비우기 시도
                            train_embeddings_batch = []
                            train_labels_batch = []


                    # 일정 간격으로 처리 현황 출력
                    if (i + 1) % 100 == 0: # 세그먼트 수가 많을 수 있으므로 간격 조정
                         print(f"    {i+1}/{len(X_train_segment_info)} 훈련 세그먼트 처리 완료 (증강 포함).")


            # 테스트 데이터 세그먼트 정보 목록을 배치 처리 (훈련 데이터와 동일한 로직)
            print("  테스트 데이터 세그먼트 처리 중...")
            test_embeddings_batch = []
            test_labels_batch = []
            # Step 1에서 반환된 테스트 세그먼트 정보 목록(X_test_segment_info) 사용
            for i, segment_info_tuple in enumerate(X_test_segment_info):
                original_file_path, start_time_sec = segment_info_tuple
                original_label_encoded = y_test_encoded[i]

                # print(f"    처리 세그먼트 {i+1}/{len(X_test_segment_info)}: 파일 '{os.path.basename(original_file_path)}' 시작 {start_time_sec:.2f}초")
                embedding = None # 임베딩 초기화

                # 테스트 세그먼트는 증강하지 않음 (augment_with_noise=False)
                try:
                     embedding = extract_func(
                          segment_info_tuple, # 세그먼트 정보 튜플 전달
                          model,
                          augment_with_noise=False, # 증강 적용 안함
                          noise_audio_paths=noise_audio_paths,
                          noise_level=0.1
                     )

                     # 임베딩 추출 성공 시 배치 목록에 추가
                     if embedding is not None:
                         test_embeddings_batch.append(embedding)
                         test_labels_batch.append(original_label_encoded)
                     else:
                         # 임베딩 추출 실패 시 경고 메시지 출력
                         print(f"      경고: 파일 '{os.path.basename(original_file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 임베딩 추출 실패 (결과 None).")

                # 임베딩 추출 중 예외 발생 시 오류 메시지 출력 및 다음 세그먼트로 이동
                except Exception as e:
                     print(f"    오류: 파일 '{os.path.basename(original_file_path)}' 세그먼트 시작 {start_time_sec:.2f}초 지점 임베딩 추출 중 예외 발생: {e}")
                     import traceback
                     traceback.print_exc(limit=2)
                     # 오류가 발생해도 루프를 중단하지 않고 다음 세그먼트로 계속 진행


                # 배치 사이즈에 도달했거나 마지막 세그먼트인 경우 현재 배치 저장
                if (len(test_embeddings_batch) >= EMBEDDING_EXTRACTION_BATCH_SIZE) or (i == len(X_test_segment_info) - 1 and test_embeddings_batch):
                    print(f"    저장 중: 테스트 배치 {len(test_batch_files) + 1}, 현재 배치 세그먼트 수: {len(test_embeddings_batch)}")
                    try:
                         # 배치 목록을 numpy 배열로 변환
                         batch_embeddings_array = np.array(test_embeddings_batch)
                         batch_labels_array = np.array(test_labels_batch)

                         # 배치 데이터를 임시 파일에 .npz 형식으로 저장
                         batch_file_path = os.path.join(temp_embedding_dir, f'{model_name}_test_batch_{len(test_batch_files)}.npz')
                         np.savez(batch_file_path, embeddings=batch_embeddings_array, labels=batch_labels_array)
                         test_batch_files.append(batch_file_path) # 저장된 파일 경로 목록에 추가

                         # 현재 배치 목록을 비우고 메모리 해제 시도
                         test_embeddings_batch = []
                         test_labels_batch = []
                         print(f"    테스트 배치 {len(test_batch_files)} 저장 완료. 메모리 해제.")
                         gc.collect() # 가비지 컬렉션 강제 실행

                    # 배치 저장 중 오류 발생 시 처리
                    except Exception as e:
                         print(f"    오류: 테스트 배치 저장 실패 - 배치 {len(test_batch_files) + 1}, 오류: {e}")
                         # 오류 발생 시에도 메모리 확보를 위해 배치 목록 비우기 시도
                         test_embeddings_batch = []
                         test_labels_batch = []


                # 일정 간격으로 처리 현황 출력
                if (i + 1) % 50 == 0: # 세그먼트 수가 많을 수 있으므로 간격 조정
                    print(f"    {i+1}/{len(X_test_segment_info)} 테스트 세그먼트 처리 완료.")


            # 현재 모델에 대해 저장된 훈련/테스트 배치 파일 경로 목록 저장
            X_train_embeddings[model_name] = train_batch_files
            X_test_embeddings[model_name] = test_batch_files

            # --- 데이터 충분성 확인 (세그먼트 기준) ---
            # 이 모델에 대해 훈련 및 테스트에 충분한 데이터(각 클래스 최소 2개)가 추출되었는지 확인합니다.
            # 이를 위해 저장된 배치 파일에서 레이블만 다시 로드하여 확인합니다.
            current_model_train_labels = []
            print(f"\n  {model_name} 訓練 배치 파일에서 레이블 수집 중 (데이터 충분성 확인용)...")
            # 저장된 훈련 배치 파일 목록을 반복
            for batch_file in train_batch_files:
                 try:
                      # .npz 파일 로드하여 레이블만 가져오기
                      with np.load(batch_file) as data:
                           current_model_train_labels.extend(data['labels']) # 레이블 목록에 추가
                 except Exception as e:
                      # 배치 파일 로드 중 오류 발생 시 처리
                      print(f"오류: {model_name} 訓練 배치 파일 로드 실패 (수집된 레이블 수 확인 중) - {batch_file}, 오류: {e}")

            current_model_test_labels = []
            print(f"  {model_name} 테스트 배치 파일에서 레이블 수집 중 (데이터 충분성 확인용)...")
            # 저장된 테스트 배치 파일 목록을 반복
            for batch_file in test_batch_files:
                 try:
                      # .npz 파일 로드하여 레이블만 가져오기
                      with np.load(batch_file) as data:
                           current_model_test_labels.extend(data['labels']) # 레이블 목록에 추가
                 except Exception as e:
                      # 배치 파일 로드 중 오류 발생 시 처리
                      print(f"오류: {model_name} 테스트 배치 파일 로드 실패 (수집된 레이블 수 확인 중) - {batch_file}, 오류: {e}")


            # 현재 모델에 대한 데이터 충분성 최종 확인
            # 훈련/테스트 모두 충분한 샘플 수와 클래스 개수(2개 이상)를 가져야 함
            if len(current_model_train_labels) >= MIN_SAMPLES_PER_CLASS_TRAIN and len(np.unique(current_model_train_labels)) >= 2 and \
               len(current_model_test_labels) >= MIN_SAMPLES_PER_CLASS_TEST and len(np.unique(current_model_test_labels)) >= 2: # 훈련/테스트 모두 각 클래스 최소 샘플 수 이상인지 확인
                 print(f"\n  {model_name} 모델에 대한 충분한 훈련 및 테스트 데이터가 추출되었습니다.")
                 # 하나 이상의 모델이 훈련 가능한 데이터를 추출했으므로 전체 플래그를 True로 설정
                 are_embeddings_extracted_successfully_any_model = True

                 # 학습/평가를 위해 이 모델의 필터링된 레이블 저장 (NumPy 배열 형태)
                 y_train_filtered[model_name] = np.array(current_model_train_labels)
                 y_test_filtered[model_name] = np.array(current_model_test_labels)
                 y_test_encoded_filtered[model_name] = y_test_filtered[model_name] # 평가 보고서용으로도 저장

                 # 임베딩 추출 및 파일 저장 완료 메시지
                 print(f"  {model_name} 임베딩 추출 및 파일 저장 완료.")
                 print(f"  훈련 임베딩 배치 파일 수: {len(X_train_embeddings[model_name])}")
                 print(f"  훈련 세그먼트 수 (총): {y_train_filtered[model_name].shape[0]}") # 세그먼트 수로 변경
                 print(f"  테스트 임베딩 배치 파일 수: {len(X_test_embeddings[model_name])}")
                 print(f"  테스트 세그먼트 수 (총): {y_test_filtered[model_name].shape[0]}") # 세그먼트 수로 변경


            else:
                 # 데이터 부족 시 경고 메시지 출력
                 print(f"\n경고: {model_name} 모델에 대한 충분한 훈련 및 테스트 데이터가 추출되지 않았습니다 (훈련/테스트 각 클래스 최소 {MIN_SAMPLES_PER_CLASS_TRAIN}/{MIN_SAMPLES_PER_CLASS_TEST}개 필요).")
                 # 해당 모델의 데이터 관련 변수들을 빈 값으로 초기화하여 이후 단계에서 사용되지 않도록 함
                 X_train_embeddings[model_name] = []
                 X_test_embeddings[model_name] = []
                 y_train_filtered[model_name] = np.array([])
                 y_test_filtered[model_name] = np.array([])
                 y_test_encoded_filtered[model_name] = np.array([])
                 # 전체 플래그(are_embeddings_extracted_successfully_any_model)는 다른 모델이 준비될 경우 True가 될 수 있음


        # 모든 모델 처리가 끝난 후 전체 임베딩 추출 단계 성공 여부 최종 판단
        if are_embeddings_extracted_successfully_any_model:
             print("\n하나 이상의 모델에 대해 임베딩 추출 및 데이터 준비 단계 성공.")
        else:
             print("\n경고: 모든 모델에서 훈련 가능한 임베딩 샘플이 부족합니다 (각 클래스 최소 2개 필요). 임베딩 추출 단계는 완료되었으나 학습/평가할 데이터가 없습니다.")


    print("\n모델별 임베딩 추출 단계 완료.")

# 임베딩 추출 함수 정의 누락 시 실행되는 else 블록
else: # 임베딩 추출 함수 정의가 누락된 경우
    print("\n임베딩 추출 함수 정의 누락으로 임베딩 추출 단계를 건너뜁니다.")
    # 이 경우, 이후 단계에서 데이터가 없으므로 관련 변수들을 빈 값으로 초기화해야 합니다.
    X_train_embeddings = {}
    X_test_embeddings = {}
    y_train_filtered = {}
    y_test_filtered = {}
    y_test_encoded_filtered = {}
    are_embeddings_extracted_successfully_any_model = False
    embedding_dims = {}
    temp_embedding_dir = None # 임시 디렉토리가 생성되지 않았으므로 cleanup 시도하지 않도록 None 설정


# --- Helper function to load batches from saved files ---
# 저장된 .npz 배치 파일에서 임베딩과 레이블을 불러오는 제너레이터 함수
# 이 함수는 임베딩 추출 블록 외부에 정의되어야 다른 단계(모델 학습/평가)에서 사용 가능합니다.
# (이 함수 정의는 이 셀 또는 별도 셀에 있어야 합니다. 여기서는 이 셀에 포함합니다.)
# 이전에 정의되지 않았다면 여기서 정의합니다.
if 'batch_generator' not in locals():
    def batch_generator(batch_file_paths):
        """Generates batches of embeddings and labels from saved .npz files."""
        # print(f"Debug: batch_generator called with {len(batch_file_paths)} files.") # 디버그 출력 (필요 시 활성화)
        if not batch_file_paths:
             # print("Debug: batch_file_paths is empty.") # 디버그 출력
             return # 파일 경로 목록이 비어 있으면 빈 제너레이터 반환

        # 각 배치 파일 경로를 반복
        for file_path in batch_file_paths:
            # print(f"Debug: Loading batch from file: {file_path}") # 디버그 출력
            try:
                # .npz 파일 로드
                with np.load(file_path) as data:
                    embeddings = data['embeddings'] # 임베딩 데이터 로드
                    labels = data['labels']       # 레이블 데이터 로드
                    # 로드된 데이터를 배치로 yield (파일 하나가 하나의 배치)
                    # print(f"Debug: Yielding batch with shape {embeddings.shape}, {labels.shape}") # 디버그 출력
                    yield embeddings, labels # 제너레이터로 데이터 반환
            # 파일 로드 중 오류 발생 시 처리
            except Exception as e:
                print(f"오류: 임베딩 배치 파일 로드 실패 - {file_path}, 오류: {e}")
                continue # 오류 발생 파일은 건너뛰고 다음 파일로 진행


# --- 4. 모델 학습을 위한 레이블 준비 (One-Hot 인코딩) ---
# 모델 학습에 사용될 레이블을 One-Hot 인코딩 형식으로 변환합니다.
y_train_one_hot = {} # 훈련 레이블 (One-Hot)
y_test_one_hot = {}  # 테스트 레이블 (One-Hot)

# 충분한 필터링된 데이터가 있는 모델들을 식별합니다.
models_ready_for_one_hot = [name for name, data in y_train_filtered.items() if data.size > 0 and len(np.unique(data)) >= 2]

is_data_ready_for_training = False # 학습을 위한 데이터 준비 완료 여부 플래그 (최종 결과)

# 하나 이상의 모델이 One-Hot 인코딩을 진행할 준비가 되었는지, 그리고 label_encoder와 num_classes가 유효한지 확인
if models_ready_for_one_hot and 'label_encoder' in locals() and label_encoder is not None and 'num_classes' in locals() and num_classes >= 2:
    print("\n모델별 레이블 One-Hot 인코딩 시작...")

    # One-Hot 인코딩 준비가 된 각 모델에 대해 처리
    for model_name in models_ready_for_one_hot:
        # 필터링된 레이블이 존재하고 클래스가 2개 이상인지 다시 한번 확인 (안전 장치)
        if model_name in y_train_filtered and y_train_filtered[model_name].size > 0 and len(np.unique(y_train_filtered[model_name])) >= 2 and \
           model_name in y_test_filtered and y_test_filtered[model_name].size > 0 and len(np.unique(y_test_filtered[model_name])) >= 2:

             print(f"--- {model_name} 레이블 인코딩 중 ---")
             try:
                 # 공유된 label_encoder와 전체 클래스 수를 사용하여 필터링된 레이블을 One-Hot 인코딩
                 y_train_one_hot[model_name] = tf.keras.utils.to_categorical(y_train_filtered[model_name], num_classes=num_classes)
                 y_test_one_hot[model_name] = tf.keras.utils.to_categorical(y_test_filtered[model_name], num_classes=num_classes)
                 print(f"  {model_name} 훈련 레이블 형태 (One-Hot): {y_train_one_hot[model_name].shape}")
                 print(f"  {model_name} 테스트 레이블 형태 (One-Hot): {y_test_one_hot[model_name].shape}")
                 is_data_ready_for_training = True # 하나 이상의 모델이 성공적으로 인코딩되면 전체 플래그를 True로 설정

             # One-Hot 인코딩 중 오류 발생 시 처리
             except Exception as e:
                 print(f"  오 오류: {model_name} 레이블 One-Hot 인코딩 중 오류 발생: {e}")
                 # 오류 발생 시 해당 모델의 One-Hot 레이블을 빈 배열로 초기화
                 y_train_one_hot[model_name] = np.array([])
                 y_test_one_hot[model_name] = np.array([])


        else: # 이 경우는 models_ready_for_one_hot 목록에 잘못 포함된 경우 (안전 장치)
            print(f"  경고: {model_name}에 대한 필터링된 레이블이 부족하거나 클래스가 2개 미만이어서 One-Hot 인코딩을 건너뜁니다 (재확인 실패).")
            # 해당 모델의 One-Hot 레이블을 빈 배열로 초기화
            y_train_one_hot[model_name] = np.array([])
            y_test_one_hot[model_name] = np.array([])


    # 모든 모델 처리가 끝난 후, 최종적으로 학습 가능한 데이터가 있는지 확인 (하나라도 성공했으면 True)
    if not any(arr.size > 0 for arr in y_train_one_hot.values()):
         print("\n일부 모델의 레이블 One-Hot 인코딩 실패 또는 데이터 부족.")
         is_data_ready_for_training = False # 학습 가능한 데이터가 전혀 없으면 최종 플래그 False


    if is_data_ready_for_training:
         print("\n모델별 레이블 One-Hot 인코딩 단계 완료: 학습/평가 진행 준비 완료.")


# 데이터 부족, 클래스 수 부족, 또는 label_encoder/num_classes 누락으로 One-Hot 인코딩 건너뛰는 경우
else:
    print("\n임베딩 추출 단계에서 충분한 데이터가 없거나, 클래스 수 부족, 또는 label_encoder 누락으로 레이블 One-Hot 인코딩을 건너뜠습니다.")
    is_data_ready_for_training = False # 학습 가능한 데이터 없음


if is_data_ready_for_training:
    print("\n데이터 로드, 임베딩 추출 및 데이터셋 준비 단계 성공: 학습/평가 진행 가능.")
else:
    print("\n데이터 로드, 임베딩 추출 및 데이터셋 준비 단계 실패: 훈련에 필요한 데이터가 부족합니다.")


# --- 5. 모델 구축, 학습 및 평가 실행 ---
# 분류 모델을 구축하고, 학습시키고, 평가하는 단계입니다.
# 필요한 함수들(build_classifier_model, train_model, evaluate_and_visualize_model)이 정의되어 있는지 확인합니다.
if 'build_classifier_model' not in locals() or 'train_model' not in locals() or 'evaluate_and_visualize_model' not in locals():
     print("오류: 필요한 모델 구축, 학습 또는 평가 함수가 정의되지 않았습니다. 모델 학습/평가를 건너뜁니다.")
     are_models_trained = False # 모델 학습 완료 여부 플래그
     trained_models = {} # 학습된 모델 저장 딕셔너리
     training_histories = {} # 학습 기록 저장 딕셔너리
     evaluation_results = {} # 평가 결과 저장 딕셔너리

# 필요한 함수들이 정의되어 있고, 학습을 위한 데이터 준비가 완료된 경우 이 else 블록 실행
else: # 함수 정의 및 데이터 준비 완료
    print("\n모델 구축, 학습 및 평가 실행 시작...")

    # 학습된 모델, 기록, 평가 결과를 저장할 딕셔너리 초기화
    trained_models = {}
    training_histories = {}
    evaluation_results = {} # 모델별 평가 지표 (손실, 정확도 등) 저장

    # 임베딩 차원 정보(embedding_dims)가 임베딩 추출 단계에서 채워졌는지 확인
    if not embedding_dims:
         print("오류: 임베딩 차원 정보가 누락되었습니다 (embedding_dims). 모델 구축/학습을 건너뜁니다.")
         are_models_trained = False # 모델 학습 플래그 False
    else: # 임베딩 차원 정보 존재
        are_models_trained = False # 모델 학습 플래그 초기화

        # 학습을 위한 데이터 준비가 완료된 경우 모델 학습 및 평가 진행
        if is_data_ready_for_training:
            print("\n모델별 분류기 구축, 학습 및 평가 진행 중...")

            # One-Hot 인코딩이 성공적으로 완료되어 학습 가능한 데이터가 있는 모델 목록
            models_to_process_keys = [name for name, data in y_train_one_hot.items() if data.size > 0]

            # 학습할 모델이 없는 경우 경고 메시지 (is_data_ready_for_training이 True면 이 경우는 발생하면 안됨)
            if not models_to_process_keys:
                print("경고: 훈련 데이터가 준비된 모델이 없습니다. 학습/평가를 건너뜁니다.")


            # 학습 가능한 각 모델에 대해 루프 실행
            for model_name in models_to_process_keys:
                print(f"\n--- {model_name} 모델 처리 중 ---")

                # 학습 및 평가에 필요한 모든 데이터와 변수가 현재 모델에 대해 유효한지 최종 확인
                # (is_data_ready_for_training이 True이고 models_to_process_keys에 포함되었다면 유효해야 함)
                if model_name in X_train_embeddings and X_train_embeddings[model_name] and \
                   model_name in y_train_one_hot and y_train_one_hot[model_name].size > 0 and \
                   model_name in X_test_embeddings and X_test_embeddings[model_name] and \
                   model_name in y_test_one_hot and y_test_one_hot[model_name].size > 0 and \
                   model_name in y_test_filtered and y_test_filtered[model_name].size > 0 and \
                   model_name in y_test_encoded_filtered and y_test_encoded_filtered[model_name].size > 0 and \
                   model_name in embedding_dims and 'num_classes' in locals() and num_classes >= 2 and \
                   'label_encoder' in locals() and label_encoder is not None:

                    # Get embedding dimension for the current model
                    # 임베딩 차원 딕셔너리에서 현재 모델의 차원 가져오기
                    embedding_dim = embedding_dims.get(model_name) # .get() 메서드 사용 (없으면 None 반환)

                    # 임베딩 차원이 유효한지 다시 확인 (안전 장치)
                    if embedding_dim is None:
                        print(f"경고: {model_name}의 임베딩 차원 정보가 유효하지 않습니다. 모델 구축/학습을 건너뜁니다.")
                        trained_models[model_name] = None
                        training_histories[model_name] = None
                        evaluation_results[model_name] = {'loss': None, 'accuracy': None}
                        continue # 현재 모델 처리를 건너뛰고 다음 모델로 이동


                    print(f"  {model_name} 임베딩 차원: {embedding_dim}")

                    # 분류기 모델 구축
                    # build_classifier_model 함수 사용
                    classifier_model = build_classifier_model(
                         input_shape=embedding_dim, # 모델 입력 형태 (임베딩 차원)
                         num_classes=num_classes, # 분류할 클래스 수
                         learning_rate=0.001 # 학습률 정의
                    )

                    # 학습 및 평가를 위한 배치 제너레이터 생성
                    # batch_generator 함수 사용 (임시 파일 경로 목록 전달)
                    train_data_generator = batch_generator(X_train_embeddings[model_name])
                    test_data_generator = batch_generator(X_test_embeddings[model_name])

                    # Calculate steps per epoch for training and validation
                    # 총 샘플 수 / 임베딩 추출 시의 배치 사이즈 (모델 학습 배치 사이즈와 같다고 가정)
                    # NOTE: This assumes the batch size used in training (train_model) is the same as EMBEDDING_EXTRACTION_BATCH_SIZE
                    # If not, this calculation needs adjustment.
                    # A simpler approach is to pass the generator directly without steps_per_epoch if the generator handles epochs.
                    # However, Keras fit with generator often requires steps_per_epoch.
                    # Let's assume training batch size is EMBEDDING_EXTRACTION_BATCH_SIZE for now for simplicity.
                    total_train_samples = y_train_filtered[model_name].shape[0] # 총 훈련 샘플 수
                    total_test_samples = y_test_filtered[model_name].shape[0]   # 총 테스트 샘플 수

                    # Ensure steps are at least 1 if there are samples
                    # 훈련 스텝 수 계산 (0이 되지 않도록 최소 1 보장)
                    train_steps_per_epoch = max(1, total_train_samples // EMBEDDING_EXTRACTION_BATCH_SIZE)
                    # 평가 스텝 수 계산 (0이 되지 않도록 최소 1 보장)
                    test_steps = max(1, total_test_samples // EMBEDDING_EXTRACTION_BATCH_SIZE)

                    # 총 샘플 수가 배치 사이즈보다 작을 경우 스텝 수가 0이 되는 것 방지 (재확인)
                    if total_train_samples > 0 and train_steps_per_epoch == 0:
                         train_steps_per_epoch = 1
                    if total_test_samples > 0 and test_steps == 0:
                         test_steps = 1


                    # --- 모델 학습 (배치 제너레이터 사용) ---
                    print(f"\n  --- {model_name} 모델 학습 시작 (배치 제너레이터 사용) ---")
                    # 조기 종료(EarlyStopping) 및 학습률 감소(ReduceLROnPlateau) 콜백 설정
                    # 필요한 콜백 클래스가 정의되어 있는지 확인 (안전 장치)
                    try:
                        early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
                        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=0.000001)
                        callbacks = [early_stopping, reduce_lr]
                    except NameError:
                         print("경고: EarlyStopping 또는 ReduceLROnPlateau 콜백 클래스가 정의되지 않았습니다 (Step 7). 콜백 없이 학습을 진행합니다.")
                         callbacks = [] # 콜백이 없으면 빈 리스트 사용

                    try:
                         # 총 훈련/테스트 샘플 수가 0보다 큰 경우에만 fit 실행
                         if total_train_samples > 0 and total_test_samples > 0:
                             # model.fit 메서드를 배치 제너레이터와 함께 사용
                             # train_model 함수 대신 직접 fit 호출 (이전 버전을 고려)
                             history = classifier_model.fit(
                                 train_data_generator, # 訓練 데이터 제너레이터
                                 steps_per_epoch=train_steps_per_epoch, # epoch당 訓練 스텝 수
                                 epochs=50, # 학습할 epoch 수 (상수 정의 가능)
                                 validation_data=test_data_generator, # 검증 데이터 제너레이터
                                 validation_steps=test_steps, # 검증 스텝 수
                                 callbacks=callbacks, # 콜백 목록
                                 verbose=1 # 학습 과정 출력 상세도
                             )
                             print(f"  {model_name} 모델 학습 완료.")
                             # 학습된 모델과 기록 저장
                             trained_models[model_name] = classifier_model
                             training_histories[model_name] = history
                             are_models_trained = True # 하나 이상의 모델이 학습되었음을 표시

                             # --- 모델 평가 (배치 제너레이터 사용) ---
                             print(f"\n  --- {model_name} 모델 평가 시작 (배치 제너레이터 사용) ---")
                             # evaluate_and_visualize_model 함수가 NumPy 배열을 기대하므로,
                             # 여기서는 제너레이터를 사용하여 예측 및 평가 지표를 직접 계산합니다.

                             print("\n  예측 수행 및 리포트 생성 중...")
                             try:
                                 # 예측을 위한 새로운 테스트 데이터 제너레이터 생성
                                 # 동일한 제너레이터를 재사용하면 상태 문제가 발생할 수 있으므로 새로 생성
                                 test_generator_for_predict = batch_generator(X_test_embeddings[model_name])
                                 # 총 테스트 샘플 수가 0보다 큰 경우에만 예측 실행
                                 if total_test_samples > 0:
                                      # 모델 예측 수행 (제너레이터 사용)
                                      y_pred_probs = classifier_model.predict(test_generator_for_predict, steps=test_steps)
                                      y_pred = np.argmax(y_pred_probs, axis=1) # 예측된 클래스 인덱스 (NumPy 배열)

                                      # 실제 레이블 (필터링된 인코딩 레이블 사용)
                                      y_true_filtered = y_test_encoded_filtered[model_name] # NumPy 배열

                                      # Ensure the number of predictions matches the number of true labels
                                      # 예측 결과 수와 실제 레이블 수가 일치하는지 확인 (중요!)
                                      if len(y_pred) != len(y_true_filtered):
                                           print(f"경고: {model_name} 예측 결과 수({len(y_pred)})와 실제 레이블 수({len(y_true_filtered)})가 일치하지 않습니다. 평가 리포트/혼동 행렬 생략.")
                                           evaluation_metrics = {'loss': None, 'accuracy': None} # 신뢰할 수 없으므로 평가 결과 N/A 처리
                                      else:
                                           # --- 분류 리포트 생성 ---
                                           print("\n  분류 리포트:")
                                           # target_names를 label_encoder에서 가져와 사용 (클래스 수 일치 확인)
                                           if label_encoder is not None and len(np.unique(y_true_filtered)) == num_classes:
                                                # classification_report 함수 사용
                                                print(classification_report(y_true_filtered, y_pred, target_names=label_encoder.classes_))
                                           else:
                                                # label_encoder 사용 불가 시 target_names 없이 리포트 출력
                                                print(classification_report(y_true_filtered, y_pred))
                                                print("  경고: 레이블 인코더 또는 클래스 불일치로 인해 target_names를 사용할 수 없습니다.")


                                           # --- 혼동 행렬 시각화 ---
                                           print("\n  혼동 행렬 시각화:")
                                           # 필터링된 테스트 데이터의 고유 클래스 수가 전체 클래스 수와 일치하는지 확인
                                           unique_test_labels_filtered = np.unique(y_true_filtered)
                                           if len(unique_test_labels_filtered) == num_classes:
                                                # 혼동 행렬 계산을 위해 실제/예측 레이블에 나타나는 모든 고유 레이블 사용
                                                all_possible_labels = np.unique(np.concatenate((y_true_filtered, y_pred)))
                                                # If label_encoder is available, use its classes order for consistent matrix
                                                if label_encoder is not None and hasattr(label_encoder, 'classes_'):
                                                    labels_for_matrix = np.arange(len(label_encoder.classes_))
                                                    # 실제 데이터에 나타나는 레이블만 포함하도록 필터링
                                                    present_labels_in_data = np.unique(np.concatenate((y_true_filtered, y_pred))).tolist()
                                                    labels_for_matrix = [l for l in labels_for_matrix if l in present_labels_in_data]

                                                    cm = confusion_matrix(y_true_filtered, y_pred, labels=labels_for_matrix) # 혼동 행렬 계산

                                                    # 혼동 행렬 크기 조정
                                                    plt.figure(figsize=(max(6, len(labels_for_matrix)), max(5, len(labels_for_matrix))))
                                                    # 눈금 레이블을 label_encoder 클래스 이름으로 설정 (필터링된 레이블 사용)
                                                    tick_labels = [label_encoder.classes_[i] for i in labels_for_matrix]
                                                    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                                                               xticklabels=tick_labels, yticklabels=tick_labels)
                                                else:
                                                     # label_encoder 사용 불가 시 레이블 없이 혼동 행렬 생성
                                                     labels_for_matrix = np.unique(np.concatenate((y_true_filtered, y_pred)))
                                                     cm = confusion_matrix(y_true_filtered, y_pred, labels=labels_for_matrix)
                                                     plt.figure(figsize=(max(6, len(labels_for_matrix)), max(5, len(labels_for_matrix))))
                                                     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
                                                plt.xlabel('예측 레이블')
                                                plt.ylabel('실제 레이블')
                                                plt.title(f'{model_name} 혼동 행렬')
                                                plt.show() # 혼동 행렬 그래프 출력
                                           else:
                                                # 필터링된 데이터에 모든 클래스가 포함되지 않아 혼동 행렬 생성 불가 시 경고
                                                print("  경고: 필터링된 테스트 데이터에 모든 클래스가 포함되지 않아 혼동 행렬을 생성할 수 없습니다.")

                                           # --- 모델 평가 (손실 및 정확도) ---
                                           # classifier_model.evaluate 메서드를 사용하여 손실 및 정확도 계산 (제너레이터 사용)
                                           loss, accuracy = classifier_model.evaluate(test_data_generator, steps=test_steps, verbose=0)
                                           evaluation_metrics = {'loss': loss, 'accuracy': accuracy} # 평가 지표 저장
                                           print(f"  테스트 세트 손실: {loss:.4f}")
                                           print(f"  테스트 세트 정확도: {accuracy:.4f}")
                                 else:
                                      # 테스트 샘플이 부족하여 예측/평가 건너뛰는 경우
                                      print("경고: 테스트 샘플이 부족하여 예측/평가를 건너뜁니다.")
                                      evaluation_metrics = {'loss': None, 'accuracy': None} # 결과 N/A 처리


                             # 예측 또는 리포트 생성 중 오류 발생 시 처리
                             except Exception as e:
                                 print(f"  오류: {model_name} 모델 예측 또는 리포트 생성 중 오류 발생: {e}")
                                 evaluation_metrics = {'loss': None, 'accuracy': None} # 평가 결과 N/A 처리


                             evaluation_results[model_name] = evaluation_metrics # 모델별 평가 결과 저장

                             # --- 학습 과정 시각화 ---
                             print("\n  학습 과정 시각화:")
                             if history is not None and hasattr(history, 'history'): # 학습 기록이 있는지 확인
                                 plt.figure(figsize=(12, 5))

                                 # 정확도 그래프
                                 plt.subplot(1, 2, 1)
                                 plt.plot(history.history.get('accuracy', []), label='훈련 정확도') # 훈련 정확도 (없으면 빈 리스트)
                                 if 'val_accuracy' in history.history: # 검증 정확도가 있으면 추가
                                      plt.plot(history.history['val_accuracy'], label='검증 정확도')
                                 plt.xlabel('에폭')
                                 plt.ylabel('정확도')
                                 plt.title(f'{model_name} 훈련 및 검증 정확도') # 그래프 제목
                                 plt.legend() # 범례 표시

                                 # 손실 그래프
                                 plt.subplot(1, 2, 2)
                                 plt.plot(history.history.get('loss', []), label='훈련 손실') # 훈련 손실 (없으면 빈 리스트)
                                 if 'val_loss' in history.history: # 검증 손실이 있으면 추가
                                      plt.plot(history.history['val_loss'], label='검증 손실')
                                 plt.xlabel('에폭')
                                 plt.ylabel('손실')
                                 plt.title(f'{model_name} 훈련 및 검증 손실') # 그래프 제목
                                 plt.legend() # 범례 표시

                                 plt.show() # 그래프 출력
                             else:
                                 print("  경고: 학습 기록(history)이 없어 그래프를 그릴 수 없습니다.")

                         # 훈련 또는 테스트 샘플이 부족하여 fit 실행을 건너뛴 경우
                         else:
                              print("경고: 훈련 또는 테스트 샘플이 부족하여 모델 학습/평가를 건너뜁니다.")
                              # 해당 모델의 학습/평가 결과를 None 또는 N/A로 표시
                              trained_models[model_name] = None
                              training_histories[model_name] = None
                              evaluation_results[model_name] = {'loss': None, 'accuracy': None}


                    # 모델 학습 중 오류 발생 시 처리
                    except Exception as e:
                         print(f"  오류: {model_name} 모델 학습 중 오류 발생: {e}")
                         # 해당 모델의 학습/평가 결과를 None 또는 N/A로 표시
                         trained_models[model_name] = None
                         training_histories[model_name] = None
                         evaluation_results[model_name] = {'loss': None, 'accuracy': None}


                # 학습/평가를 위한 데이터 또는 변수가 부족하여 현재 모델 처리를 건너뛰는 경우 (안전 장치)
                else:
                    print(f"경고: {model_name} 모델 학습/평가를 위한 데이터 또는 필수 변수가 부족합니다 (재확인 실패). 건너뜁니다.")
                    # 해당 모델의 학습/평가 결과를 None 또는 N/A로 표시
                    trained_models[model_name] = None
                    training_histories[model_name] = None
                    evaluation_results[model_name] = {'loss': None, 'accuracy': None}


            print("\n모델 구축, 학습 및 평가 실행 단계 완료.")

        # 데이터 준비 실패로 모델 학습 및 평가를 건너뛰는 경우
        else:
            print("\n데이터 준비 실패로 모델 학습 및 평가를 건너뜁니다.")
            are_models_trained = False # 모델 학습 플래그 False


# --- 임시 임베딩 디렉토리 정리 ---
# 임베딩 배치 파일이 저장되었던 임시 디렉토리를 삭제합니다.
# temp_embedding_dir이 유효하고 디렉토리가 실제로 존재하는 경우에만 삭제 시도
if temp_embedding_dir and os.path.exists(temp_embedding_dir):
    print(f"\n임시 임베딩 디렉토리 삭제 중: {temp_embedding_dir}")
    try:
        shutil.rmtree(temp_embedding_dir) # 디렉토리 및 내용물 모두 삭제
        print("임시 디렉토리 삭제 완료.")
    except Exception as e:
        print(f"오류: 임시 디렉토리 삭제 실패: {e}")


# --- 6. 모델 성능 비교 (요약) ---
print("\n6. 모델 성능 비교 (요약) 중...")
print("\n--- 모델별 최종 성능 비교 ---")
# 평가 결과가 있고, 하나 이상의 모델이 정확도 값이 있는 경우에만 요약 출력
if evaluation_results and any(metrics.get('accuracy') is not None for metrics in evaluation_results.values()):
    # 정확도 기준으로 결과 정렬 (선택 사항)
    # 정확도 값이 None인 경우는 -1로 처리하여 뒤로 보내도록 함
    sorted_results = sorted(evaluation_results.items(), key=lambda item: item[1].get('accuracy') if item[1].get('accuracy') is not None else -1, reverse=True)

    # 정렬된 결과 출력
    for model_name, metrics in sorted_results:
        print(f"  {model_name}:")
        print(f"    테스트 손실: {metrics.get('loss'):.4f}" if metrics.get('loss') is not None else "    테스트 손실: N/A") # 손실 값 출력 (없으면 N/A)
        print(f"    테스트 정확도: {metrics.get('accuracy'):.4f}" if metrics.get('accuracy') is not None else "    테스트 정확도: N/A") # 정확도 값 출력 (없으면 N/A)
else:
    print("평가 결과가 없어 모델 성능 비교를 수행할 수 없습니다.")


# --- 7. 새로운 오디오 파일에 대한 예측 (예시) ---
print("\n7. 새로운 오디오 파일 예측 예시 중...")
print("\n--- 새로운 오디오 파일 예측 예시 ---")

# 학습된 모델이 하나라도 있는 경우에만 예측 예시 진행
if any(model is not None for model in trained_models.values()):
    # 예측에 사용할 학습된 모델 선택 (예: 첫 번째 학습된 모델)
    model_to_predict_name = None
    for name, model in trained_models.items():
         if model is not None:
              model_to_predict_name = name # 학습된 모델 이름을 찾으면 저장
              break # 첫 번째 학습된 모델만 사용 (예시 목적)
    model_to_predict = trained_models.get(model_to_predict_name) # 해당 이름으로 학습된 모델 객체 가져오기


    # 예측에 사용할 모델이 유효한 경우 예측 진행
    # 예측에는 YAMNet 임베딩 함수와 YAMNet 모델 객체가 필요합니다.
    if model_to_predict is not None and \
       'extract_yamnet_embedding' in locals() and callable(extract_yamnet_embedding) and \
       'loaded_models' in locals() and loaded_models.get('YAMNet') is not None and \
       'label_encoder' in locals() and label_encoder is not None: # label_encoder가 정의되어 있어야 함

        print(f"\n예측에 사용할 모델: {model_to_predict_name}")

        # 예측할 샘플 오디오 파일 경로 지정
        # Step 1에서 로드된 원본 데이터 경로 목록(all_audio_paths) 사용 시도
        predict_audio_path = None

        # 원본 데이터 경로 목록(all_audio_paths)이 있고 비어있지 않으면 첫 번째 경로 사용
        if 'all_audio_paths' in locals() and all_audio_paths:
            predict_audio_path = all_audio_paths[0]
            print(f"예측을 위해 데이터셋에서 샘플 파일 선택: {predict_audio_path}")
        # DeepShip 기본 경로가 정의되어 있고, 해당 경로에 예시 파일(103.wav)이 존재하면 사용
        elif 'DEEPSHIP_BASE_PATH' in locals() and os.path.exists(os.path.join(DEEPSHIP_BASE_PATH, 'Cargo', '103.wav')):
            predict_audio_path = os.path.join(DEEPSHIP_BASE_PATH, 'Cargo', '103.wav')
            print(f"예측을 위해 DeepShip에서 샘플 파일 선택: {predict_audio_path}")
        # 다른 폴백 경로나 더미 파일 생성 로직을 여기에 추가 가능

        # predict_on_new_audio 함수가 정의되어 있는지 확인하고 예측 실행
        if predict_audio_path and os.path.exists(predict_audio_path):
             if 'predict_on_new_audio' not in locals() or not callable(predict_on_new_audio):
                  print("오류: 'predict_on_new_audio' 함수가 정의되지 않았습니다 (Step 8). 예측을 건너뜁니다.")
             else:
                 try:
                      # predict_on_new_audio 함수 호출
                      # 학습된 분류기 모델, 전처리 및 임베딩 추출 함수(YAMNet 사용), 원본 YAMNet 모델 객체, label_encoder, 예측 파일 경로 전달
                      # predict_on_new_audio 함수는 내부적으로 YAMNet 임베딩 추출 함수를 호출합니다.
                      # 이 함수는 파일 경로를 받도록 설계되어 있다고 가정합니다. (세그먼트 처리와 분리)
                      # 만약 predict_on_new_audio 함수도 세그먼트 기반으로 변경해야 한다면 수정이 필요합니다.
                      # 현재는 파일 기반 예측 함수가 Step 8에 정의되어 있다고 가정합니다.
                      predicted_label = predict_on_new_audio(
                           classifier_model=model_to_predict, # 학습된 분류기 모델
                           embedding_extractor_func=extract_yamnet_embedding, # 사용할 임베딩 추출 함수 (YAMNet)
                           embedding_model=loaded_models.get('YAMNet'), # 임베딩 모델 객체 (YAMNet)
                           label_encoder=label_encoder, # LabelEncoder 객체
                           audio_path=predict_audio_path, # 예측 파일 경로
                           # extract_yamnet_embedding 함수가 필요로 하는 다른 인자들 (e.g., target_sample_rate)도 predict_on_new_audio 함수를 통해 전달되어야 함
                           # 현재 predict_on_new_audio 함수가 이러한 인자를 어떻게 처리하는지 불분명하므로 기본값 사용 또는 수정 필요
                           target_sample_rate=TARGET_SAMPLE_RATE # 전역 상수를 사용하거나 predict_on_new_audio 인자로 전달
                      )
                      # 예측 결과는 함수 내부에서 출력됩니다.
                 except Exception as e:
                      print(f"오류: 새로운 오디오 파일 예측 중 예외 발생: {e}")
                      import traceback
                      traceback.print_exc(limit=2)

        else:
            print("\n예측을 수행할 수 없습니다: 예측할 오디오 파일을 찾을 수 없습니다.")


    else:
        print("\n예측을 수행할 수 없습니다:")
        if any(model is None for model in trained_models.values()):
             print("  학습된 모델이 없어 예측을 수행할 수 없습니다.")
        if 'extract_yamnet_embedding' not in locals() or not callable(extract_yamnet_embedding):
             print("  YAMNet 임베딩 추출 함수가 정의되지 않았습니다.")
        if 'loaded_models' not in locals() or loaded_models.get('YAMNet') is None:
             print("  YAMNet 임베딩 모델이 로드되지 않았습니다.")
        if 'label_encoder' not in locals() or label_encoder is None:
             print("  LabelEncoder가 없습니다.")


# 학습된 모델이 전혀 없는 경우 예측 예시 건너뛰기
else:
    print("\n학습된 모델이 없어 예측을 수행할 수 없습니다.")


print("\n모델별 전체 파이프라인 실행 및 결과 비교 단계 완료.")

# 이 셀의 모든 코드가 성공적으로 실행되었음을 나타냅니다.
# 하지만 실제 파이프라인의 성공 여부는 중간 로그 및 최종 평가 결과를 확인해야 합니다.
# 특히 "데이터 부족" 메시지나 다른 오류가 없었는지 확인하세요.

## 7. 모델 구축 및 학습 함수 정의

각 모델에서 추출된 임베딩을 입력으로 받아 최종 분류를 수행하는 Keras 모델(분류 헤드)을 구축하고 학습시키는 함수를 정의합니다.

In [ ]:
import tensorflow as tf # Ensure tf is imported
from tensorflow.keras.models import Model # Ensure Model is imported
from tensorflow.keras.layers import Input, Dense, Dropout # Ensure layers are imported
from tensorflow.keras.optimizers import Adam # Ensure Adam is imported
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau # Ensure callbacks are imported
import numpy as np # Ensure numpy is imported

print("\n7. 분류기 모델 구축 및 학습 함수 정의 중...")

def build_classifier_model(input_shape, num_classes, learning_rate=0.001):
    """
    임베딩 입력을 위한 분류 헤드 Keras 모델을 구축하고 컴파일합니다.

    Args:
        input_shape (int): 입력 임베딩의 차원.
        num_classes (int): 출력 클래스의 수 (이진 분류의 경우 2).
        learning_rate (float): Adam 옵티마이저의 학습률.

    Returns:
        tensorflow.keras.models.Model: 컴파일된 Keras 모델 객체.
    """
    embedding_input = Input(shape=(input_shape,), name='embedding_input')

    x = Dense(128, activation='relu')(embedding_input)
    x = Dropout(0.4)(x) # Increased dropout for regularization
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.4)(x) # Increased dropout for regularization
    # Output layer for binary classification (num_classes=2) or multi-class
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=embedding_input, outputs=output)

    # Model compilation
    optimizer = Adam(learning_rate=learning_rate)

    # Use categorical_crossentropy loss for multi-class (including binary with 2 classes)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    print("  분류 헤드 모델 구축 및 컴파일 완료.")
    model.summary()

    return model


def train_model(model, X_train, y_train_one_hot, X_test, y_test_one_hot,
                               epochs=50, batch_size=16):
    """
    주어진 데이터로 Keras 모델을 학습시킵니다.

    Args:
        model (tensorflow.keras.models.Model): 학습할 컴파일된 Keras 모델.
        X_train (np.ndarray): 訓練 임베딩 데이터.
        y_train_one_hot (np.ndarray): One-Hot 인코딩된 訓練 레이블.
        X_test (np.ndarray): 테스트 임베딩 데이터 (검증에 사용).
        y_test_one_hot (np.ndarray): One-Hot 인코딩된 테스트 레이블 (검증에 사용).
        epochs (int): 학습 에폭 수.
        batch_size (int): 배치 크기.

    Returns:
        tuple: (trained_model, history) 학습된 모델 객체와 학습 기록.
               データ不足またはエラー発生時 (None, None) 返還。
    """
    print("\n  모델 학습 시작...")

    # Data validation and sufficiency checks
    if X_train.size == 0 or y_train_one_hot.size == 0 or X_test.size == 0 or y_test_one_hot.size == 0:
        print("  경고: 분류기 학습을 위한 훈련 또는 테스트 데이터가 부족합니다. 학습을 건너뜁니다.")
        return None, None

    if X_train.shape[0] < 2 or X_test.shape[0] < 2:
        print("  경고: 분류기 학습을 위한 훈련 또는 테스트 데이터 샘플 수가 2개 미만입니다. 학습을 건너뜁니다.")
        return None, None

    # Ensure there are at least 2 unique classes in the training labels
    # np.unique(np.argmax(y_train_one_hot, axis=1)) gets the unique original encoded labels
    if y_train_one_hot.ndim > 1 and len(np.unique(np.argmax(y_train_one_hot, axis=1))) < 2:
         print("  경고: 분류기 학습을 위한 훈련 레이블에 클래스가 2개 미만입니다. 학습을 건너뜁니다.")
         return None, None
    elif y_train_one_hot.ndim == 1 and len(np.unique(y_train_one_hot)) < 2: # Handle case where labels might not be one-hot yet (should be one-hot here)
         print("  경고: 분류기 학습을 위한 훈련 레이블에 클래스가 2개 미만입니다 (One-Hot 인코딩 전 상태 확인). 학습을 건너뜁니다.")
         return None, None


    # EarlyStopping 및 ReduceLROnPlateau 콜백 설정
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=0.000001)
    callbacks = [early_stopping, reduce_lr]

    # Model training
    try:
        history = model.fit(
            X_train, y_train_one_hot,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_test, y_test_one_hot), # Use validation_data for test set evaluation during training
            callbacks=callbacks,
            verbose=1
        )
        print("  모델 학습 완료.")
        return model, history
    except Exception as e:
        print(f"  오류: 모델 학습 중 오류 발생: {e}")
        return None, None


print("\n모델 구축 및 학습 함수 정의 완료.")

## 8. 모델 평가 및 결과 시각화 함수 정의

학습된 모델의 성능을 테스트 데이터에 대해 평가하고, 분류 리포트, 혼동 행렬, 학습 곡선 등을 시각화하는 함수를 정의합니다.

In [ ]:
import matplotlib.pyplot as plt # Ensure matplotlib is imported
import seaborn as sns # Ensure seaborn is imported
from sklearn.metrics import classification_report, confusion_matrix # Ensure sklearn metrics are imported
import numpy as np # Ensure numpy is imported

print("\n8. 모델 평가 및 결과 시각화 함수 정의 중...")

# Assuming necessary libraries (matplotlib, seaborn, sklearn.metrics, numpy) are imported.
# Assuming label_encoder is available if evaluation is attempted.

def evaluate_and_visualize_model(model_name, trained_model, X_test, y_test_one_hot, y_test_encoded_filtered,
                                 label_encoder, history):
    """
    학습된 모델의 성능을 평가하고 결과를 시각화합니다.

    Args:
        model_name (str): 모델 이름 (예: 'YAMNet', 'VGGish').
        trained_model (tf.keras.Model): 학습된 모델 객체.
        X_test (np.ndarray): 테스트 임베딩 데이터.
        y_test_one_hot (np.ndarray): One-Hot 인코딩된 테스트 레이블.
        y_test_encoded_filtered (np.ndarray): 필터링된 실제 인코딩 레이블 (혼동 행렬, 리포트용).
        label_encoder (sklearn.preprocessing.LabelEncoder): 레이블 인코더 객체.
        history (tf.keras.callbacks.History): 모델 학습 기록.

    Returns:
        dict: 평가 결과를 담고 있는 딕셔너리 (loss, accuracy).
              평가 실패 시 None 값을 가질 수 있습니다.
    """
    print(f"\n--- {model_name} 모델 성능 평가 및 시각화 시작 ---")
    evaluation_metrics = {'loss': None, 'accuracy': None} # Initialize metrics dictionary

    # --- 1. 데이터 및 모델 유효성 확인 ---
    # Check if the model and data are valid and not empty
    if trained_model is None:
        print(f"  경고: {model_name} 모델이 학습되지 않았습니다. 평가를 건너뜁니다.")
        return evaluation_metrics

    if not isinstance(X_test, np.ndarray) or X_test.size == 0:
        print(f"  경고: {model_name} 모델의 테스트 임베딩 데이터가 유효하지 않거나 비어 있습니다. 평가를 건너뜁니다.")
        return evaluation_metrics

    if not isinstance(y_test_one_hot, np.ndarray) or y_test_one_hot.size == 0:
         print(f"  경고: {model_name} 모델의 One-Hot 테스트 레이블이 유효하지 않거나 비어 있습니다. 평가를 건너뜁니다.")
         return evaluation_metrics

    if not isinstance(y_test_encoded_filtered, np.ndarray) or y_test_encoded_filtered.size == 0:
         print(f"  경고: {model_name} 모델의 필터링된 실제 인코딩 레이블이 유효하지 않거나 비어 있습니다. 평가를 건너뜁니다.")
         return evaluation_metrics

    if label_encoder is None:
        print(f"  경고: 레이블 인코더가 None입니다. 평가 리포트 및 혼동 행렬 레이블이 정확하지 않을 수 있습니다.")
        # Proceed with evaluation but without accurate labels

    # Ensure there are at least 2 unique classes in the filtered test labels for meaningful evaluation
    unique_test_labels = np.unique(y_test_encoded_filtered)
    if len(unique_test_labels) < 2:
         print(f"  경고: 필터링된 테스트 데이터에 클래스가 2개 미만입니다 ({len(unique_test_labels)}개). 평가를 건너뜁니다.")
         return evaluation_metrics

    # Determine the number of classes from the one-hot labels shape or label_encoder
    num_classes = y_test_one_hot.shape[1] if y_test_one_hot.ndim > 1 else 1
    if label_encoder is not None:
        num_classes_from_encoder = len(label_encoder.classes_)
        if num_classes != num_classes_from_encoder:
             print(f"  경고: One-Hot 레이블 형태와 LabelEncoder의 클래스 수가 일치하지 않습니다 ({num_classes} vs {num_classes_from_encoder}).")
             # Use the number from one-hot labels for consistency with model output
             pass # Continue, using num_classes from one-hot shape


    print("  데이터 및 모델 유효성 확인 완료. 평가 진행.")

    # --- 2. 모델 평가 ---
    print("  테스트 데이터로 모델 평가...")
    try:
        loss, accuracy = trained_model.evaluate(X_test, y_test_one_hot, verbose=0)
        print(f"  테스트 세트 손실: {loss:.4f}")
        print(f"  테스트 세트 정확도: {accuracy:.4f}")
        evaluation_metrics['loss'] = loss
        evaluation_metrics['accuracy'] = accuracy
    except Exception as e:
         print(f"  오류: {model_name} 모델 평가 중 오류 발생: {e}")
         # Continue to prediction and reporting if evaluation fails but prediction might work


    # --- 3. 예측 및 리포트 생성 ---
    print("\n  예측 수행 및 리포트 생성...")
    try:
        y_pred_probs = trained_model.predict(X_test)
        y_pred = np.argmax(y_pred_probs, axis=1)

        print("\n  분류 리포트:")
        # Use target_names from label_encoder if available and matches unique test labels
        if label_encoder is not None and len(unique_test_labels) == num_classes:
             print(classification_report(y_test_encoded_filtered, y_pred, target_names=label_encoder.classes_))
        else:
             # Fallback without target names
             print(classification_report(y_test_encoded_filtered, y_pred))
             print("  경고: 레이블 인코더 또는 클래스 불일치로 인해 target_names를 사용할 수 없습니다.")


        # --- 4. 혼동 행렬 시각화 ---
        print("\n  혼동 행렬 시각화:")
        # Ensure the unique classes in the filtered test data match the number of classes for the matrix size
        if len(unique_test_labels) == num_classes:
             # Ensure labels for confusion matrix calculation cover all unique predicted and true labels
             all_possible_labels = np.unique(np.concatenate((y_test_encoded_filtered, y_pred)))
             # If label_encoder is available, use its classes order for consistent matrix
             if label_encoder is not None and hasattr(label_encoder, 'classes_'):
                 labels_for_matrix = np.arange(len(label_encoder.classes_))
                 # Filter labels_for_matrix to only include those present in y_test_encoded_filtered or y_pred for smaller datasets
                 # This avoids plotting empty rows/columns if not all classes are in the test set (post-filtering)
                 present_labels_in_data = np.unique(np.concatenate((y_test_encoded_filtered, y_pred))).tolist()
                 labels_for_matrix = [l for l in labels_for_matrix if l in present_labels_in_data]

                 cm = confusion_matrix(y_test_encoded_filtered, y_pred, labels=labels_for_matrix)

                 plt.figure(figsize=(max(6, len(labels_for_matrix)), max(5, len(labels_for_matrix)))) # Adjust figure size based on num classes
                 # Use label_encoder classes for tick labels if available, filtered to present labels
                 tick_labels = [label_encoder.classes_[i] for i in labels_for_matrix]
                 sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                            xticklabels=tick_labels, yticklabels=tick_labels)
             else:
                  # Fallback without labels if label_encoder or classes don't match
                  # Use unique labels from the data for matrix labels if label_encoder is not fully usable
                  labels_for_matrix = np.unique(np.concatenate((y_test_encoded_filtered, y_pred)))
                  cm = confusion_matrix(y_test_encoded_filtered, y_pred, labels=labels_for_matrix)
                  plt.figure(figsize=(max(6, len(labels_for_matrix)), max(5, len(labels_for_matrix)))) # Adjust figure size
                  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues') # Fallback without labels
             plt.xlabel('예측 레이블')
             plt.ylabel('실제 레이블')
             plt.title(f'{model_name} 혼동 행렬')
             plt.show()
        else:
             print("  경고: 필터링된 테스트 데이터에 모든 클래스가 포함되지 않아 혼동 행렬을 생성할 수 없습니다.")


    except Exception as e:
        print(f"  오류: {model_name} 모델 예측 또는 리포트 생성 중 오류 발생: {e}")


    # --- 5. 학습 과정 시각화 ---
    print("\n  학습 과정 시각화:")
    # Check if history object is valid and has the 'history' attribute
    if history is not None and hasattr(history, 'history'):
        plt.figure(figsize=(12, 5))

        # Accuracy plot
        plt.subplot(1, 2, 1)
        plt.plot(history.history.get('accuracy', []), label='훈련 정확도') # Use .get() for safety
        if 'val_accuracy' in history.history: # Check if validation accuracy exists
             plt.plot(history.history['val_accuracy'], label='검증 정확도')
        plt.xlabel('에폭')
        plt.ylabel('정확도')
        plt.title(f'{model_name} 훈련 및 검증 정확도')
        plt.legend()

        # Loss plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history.get('loss', []), label='훈련 손실') # Use .get() for safety
        if 'val_loss' in history.history: # Check if validation loss exists
             plt.plot(history.history['val_loss'], label='검증 손실')
        plt.xlabel('에폭')
        plt.ylabel('손실')
        plt.title(f'{model_name} 훈련 및 검증 손실')
        plt.legend()

        plt.show()
    else:
        print("  경고: 학습 기록(history)이 없어 그래프를 그릴 수 없습니다.")


    print(f"\n--- {model_name} 모델 성능 평가 및 시각화 완료 ---")
    return evaluation_metrics # Return evaluation results


print("\n모델 평가 및 결과 시각화 함수 정의 완료.")

## 9. 예측 함수 정의

학습된 모델을 사용하여 새로운 오디오 파일에 대한 예측을 수행하는 함수를 정의합니다.

In [ ]:
import os # Ensure os is imported
import numpy as np # Ensure numpy is imported

print("\n9. 예측 함수 정의 중...")

def predict_on_new_audio(model, yamnet_model, label_encoder, audio_path):
    """
    새로운 오디오 파일에 대해 학습된 모델로 예측을 수행합니다.

    Args:
        model (tensorflow.keras.models.Model): 예측에 사용할 학습된 Keras 모델.
        yamnet_model: YAMNet 모델 객체 (오디오 전처리 및 임베딩 추출에 사용).
        label_encoder (sklearn.preprocessing.LabelEncoder): 레이블 인코더 객체.
        audio_path (str): 예측할 오디오 파일 경로.

    Returns:
        str: 예측된 레이블 문자열, 또는 예측 실패 시 None.
    """
    print(f"\n새 오디오 파일('{os.path.basename(audio_path)}') 예측 시작...")

    if not os.path.exists(audio_path):
        print(f"오류: 예측할 오디오 파일을 찾을 수 없습니다: {audio_path}")
        return None

    # Use the defined YAMNet embedding extraction function for preprocessing
    # Ensure extract_yamnet_embedding is defined and available
    if 'extract_yamnet_embedding' not in locals():
        print("오류: 'extract_yamnet_embedding' 함수가 정의되지 않았습니다. 예측을 건너뜁니다.")
        return None

    new_audio_embedding = extract_yamnet_embedding(audio_path, yamnet_model)

    if new_audio_embedding is None:
        print("새 오디오 파일 임베딩 추출에 실패했습니다.")
        return None

    # Reshape the embedding to match the model's expected input shape (add batch dimension)
    new_audio_embedding = np.expand_dims(new_audio_embedding, axis=0)

    # Perform prediction
    try:
        prediction_probs = model.predict(new_audio_embedding)[0]
        predicted_class_idx = np.argmax(prediction_probs)

        # Convert predicted index back to label using the label encoder
        if label_encoder is not None and hasattr(label_encoder, 'inverse_transform'):
            predicted_label = label_encoder.inverse_transform([predicted_class_idx])[0]
        else:
            print("경고: label_encoder가 없거나 inverse_transform 메서드를 사용할 수 없습니다. 예측된 인덱스만 반환합니다.")
            predicted_label = predicted_class_idx # Return index if label_encoder is not usable


        print(f"\n예측 결과 for '{os.path.basename(audio_path)}':")
        if label_encoder is not None and hasattr(label_encoder, 'classes_'):
            # Print probabilities for each class
            for i, class_name in enumerate(label_encoder.classes_):
                print(f"  {class_name}: {prediction_probs[i]*100:.2f}%")
        else:
             # Print probabilities by index if class names are not available
             for i in range(len(prediction_probs)):
                  print(f"  Class {i} (Index {i}): {prediction_probs[i]*100:.2f}%")

        print(f"최종 예측: {predicted_label}")
        return predicted_label

    except Exception as e:
        print(f"오류: 예측 수행 중 오류 발생: {e}")
        return None


print("예측 함수 정의 완료.")

## 10. 모델별 전체 파이프라인 실행 및 결과 비교

정의된 함수들을 사용하여 각 모델(YAMNet, PANNs, VGGish)에 대해 데이터 준비(임베딩 추출 포함), 학습, 평가 과정을 순차적으로 실행하고, 최종 성능을 비교합니다.

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os # Ensure os is imported
import matplotlib.pyplot as plt # Ensure matplotlib is imported
import seaborn as sns # Ensure seaborn is imported
from sklearn.metrics import classification_report, confusion_matrix # Ensure sklearn metrics are imported
import random # Ensure random is imported for augmentation
import tempfile # Import tempfile for creating temporary directories
import shutil # Import shutil for removing directories
import sys # Import sys for checking memory usage (approximate)
import gc # Import garbage collector


print("\n10. 모델별 전체 파이프라인 실행 및 결과 비교 시작...")

# --- IMPORTANT: Ensure Step 4 (cell defining extract_... functions) is executed first! ---
print("\n" + "="*80)
print(">>>         >>>  필수 작업 안내  <<<         <<<")
print(">>>                                           <<<")
print(">>>   오디오 전처리 및 임베딩 추출 함수가 정의된 Step 4 셀을 먼저 실행했는지 확인하세요! <<<")
print(">>>   (예: 셀 ID 617c87ca) 그렇지 않으면 'Quality must be one of...' 오류가 발생합니다. <<<")
print(">>>                                           <<<")
print(">>>   DeepShip('/content/DeepShip') 및 노이즈('/content/MBARI_noise_data') 디렉토리에 <<<")
print(">>>   훈련/테스트에 필요한 오디오 파일이 충분히 있는지 확인하세요!                     <<<")
print(">>>   (각 클래스 최소 2개 파일 필요) 데이터 부족 시 '데이터 부족' 오류가 발생합니다. <<<")
print(">>>                                           <<<")
print(">>>         >>>  필수 작업 완료 후 이 셀을 실행하세요.  <<<         <<<")
print("="*80 + "\n")


# --- 1. 데이터 로드 및 준비 ---
# Calls the function defined in Step 3 (corrected version)
# Ensure load_and_prepare_dataset is defined and available
if 'load_and_prepare_dataset' not in locals():
    print("오류: 'load_and_prepare_dataset' 함수가 정의되지 않았습니다. 파이프라인 실행을 중단합니다.")
    is_data_prepared = False
    X_train_paths, X_test_paths, y_train_encoded, y_test_encoded, label_encoder, num_classes, noise_audio_paths = [], [], np.array([]), np.array([]), None, 0, [] # Initialize noise_audio_paths
else:
    # Modified to also return noise_audio_paths
    X_train_paths, X_test_paths, y_train_encoded, y_test_encoded, label_encoder, is_data_prepared, num_classes, noise_audio_paths = load_and_prepare_dataset(
        deepship_path=DEEPSHIP_BASE_PATH,
        noise_data_dir=MBARI_NOISE_BASE_DIR,
        deepship_classes=DEEPSHIP_CLASSES # Pass the global constant
    )

# --- 2. 오디오 모델 로드 ---
# Calls the function defined in Step 5
# Ensure load_audio_models is defined and available
if 'load_audio_models' not in locals():
     print("오류: 'load_audio_models' 함수가 정의되지 않았습니다. 파이프라인 실행을 중단합니다.")
     are_models_loaded = False
     loaded_models = {}
else:
    loaded_models, are_models_loaded = load_audio_models()


# --- 3. Model-wise Embedding Extraction and Data Preparation (Batch Processing & Saving) ---
# Instead of storing all embeddings in memory, we'll save them to temporary files.
temp_embedding_dir = None
are_embeddings_extracted_successfully = False
embedding_dims = {} # Dictionary to store embedding dimensions

# Ensure extract_yamnet_embedding, extract_panns_embedding, extract_vggish_embedding are defined
if 'extract_yamnet_embedding' not in locals() or 'extract_panns_embedding' not in locals() or 'extract_vggish_embedding' not in locals():
    print("오류: 임베딩 추출 함수 중 하나 이상이 정의되지 않았습니다. 임베딩 추출을 건너뜁니다.")
    # Skip the rest of the embedding extraction block
    pass
else: # All necessary functions are defined

    if is_data_prepared and are_models_loaded:
        print("\n모델별 임베딩 추출 및 데이터 준비 시작 (배치 처리 및 파일 저장 - 시간이 오래 걸릴 수 있습니다)...")

        # Create a temporary directory to store embeddings
        temp_embedding_dir = tempfile.mkdtemp()
        print(f"임시 임베딩 저장 디렉토리: {temp_embedding_dir}")

        # Dictionary mapping model names to their loaded model object, embedding dimension, and extraction function
        # Ensure embedding dimension constants and loaded models are available
        models_info = {}
        if loaded_models.get('YAMNet') is not None and 'YAMNET_EMBEDDING_DIM' in locals():
             models_info['YAMNet'] = {'model': loaded_models['YAMNet'], 'extract_func': extract_yamnet_embedding, 'dim': YAMNET_EMBEDDING_DIM}
        else:
             print("경고: YAMNet 모델 정보 또는 함수가 누락되었습니다. YAMNet 처리를 건너뜁니다.")
        if loaded_models.get('PANNs') is not None and 'PANNS_EMBEDDING_DIM' in locals():
             models_info['PANNs'] = {'model': loaded_models['PANNs'], 'extract_func': extract_panns_embedding, 'dim': PANNS_EMBEDDING_DIM}
        else:
              print("경고: PANNs 모델 정보 또는 함수가 누락되었습니다. PANNs 처리를 건너뜁니다.")
        if loaded_models.get('VGGish') is not None and 'VGGISH_EMBEDDING_DIM' in locals():
             models_info['VGGish'] = {'model': loaded_models['VGGish'], 'extract_func': extract_vggish_embedding, 'dim': VGGISH_EMBEDDING_DIM}
        else:
              print("경고: VGGish 모델 정보 또는 함수가 누락되었습니다. VGGish 처리를 건너뜁니다.")


        # Define batch size for embedding extraction (further reduced)
        EMBEDDING_EXTRACTION_BATCH_SIZE = 1 # Further reduced batch size for lower memory usage

        if models_info: # Proceed only if at least one model info is available
            all_models_extracted_at_least_one_sample = False # Track if any model extracted any sample

            for model_name, info in models_info.items():
                model = info['model']
                extract_func = info['extract_func']
                embedding_dims[model_name] = info['dim'] # Store embedding dimension

                print(f"\n--- {model_name} 임베딩 추출 중 (배치 크기: {EMBEDDING_EXTRACTION_BATCH_SIZE}) ---")
                train_batch_files = [] # List to store paths of saved train embedding batches
                test_batch_files = []  # List to store paths of saved test embedding batches

                # Process training data paths in batches
                print("  훈련 데이터 처리 중...")
                train_embeddings_batch = []
                train_labels_batch = []
                # Added index tracking for original samples
                original_train_indices = []


                for i, path in enumerate(X_train_paths):
                    original_label_encoded = y_train_encoded[i]
                    original_label_str = label_encoder.inverse_transform([original_label_encoded])[0] if label_encoder is not None and hasattr(label_encoder, 'inverse_transform') else str(original_label_encoded)

                    print(f"    처리 파일 {i+1}/{len(X_train_paths)}: {os.path.basename(path)}")

                    # Extract original embedding (no augmentation for the base sample)
                    # Ensure noise_audio_paths is passed even if augment_with_noise is False, as the function expects it.
                    try:
                         embedding = extract_func(path, model, augment_with_noise=False, noise_audio_paths=noise_audio_paths)
                         if embedding is not None:
                              train_embeddings_batch.append(embedding)
                              train_labels_batch.append(original_label_encoded)
                              original_train_indices.append(i) # Store original index
                         else:
                              print(f"      경고: 파일 '{os.path.basename(path)}' 임베딩 추출 실패 (결과 None).")

                    except Exception as e:
                         print(f"    오류: 파일 '{os.path.basename(path)}' 임베딩 추출 중 예외 발생: {e}")
                         import traceback
                         traceback.print_exc(limit=2, file=sys.stdout) # Print limited traceback
                         # Continue to next file even on error


                    # Add augmented samples for 'ship' class if noise data is available and augmentation is desired
                    # Augmentation is done here during extraction and saved as separate samples
                    if original_label_str == 'ship' and noise_audio_paths:
                         print(f"      증강 적용 중 (노이즈 혼합) for '{os.path.basename(path)}'")
                         try:
                              augmented_embedding = extract_func(
                                   path,
                                   model,
                                   augment_with_noise=True, # Apply augmentation
                                   noise_audio_paths=noise_audio_paths,
                                   noise_level=0.1
                              )
                              if augmented_embedding is not None:
                                   train_embeddings_batch.append(augmented_embedding)
                                   train_labels_batch.append(original_label_encoded) # Augmented sample keeps original label
                                   # No need to track augmented sample indices separately for now
                              else:
                                   print(f"      경고: 파일 '{os.path.basename(path)}' 증강 임베딩 추출 실패 (결과 None).")

                         except Exception as e:
                              print(f"    오류: 파일 '{os.path.basename(path)}' 증강 임베딩 추출 중 예외 발생: {e}")
                              import traceback
                              traceback.print_exc(limit=2, file=sys.stdout) # Print limited traceback
                              # Continue to next file even on error


                    # Save batch and clear lists if batch size is reached or it's the last sample
                    if (len(train_embeddings_batch) >= EMBEDDING_EXTRACTION_BATCH_SIZE) or (i == len(X_train_paths) - 1 and train_embeddings_batch):
                        print(f"    저장 중: 훈련 배치 {len(train_batch_files) + 1}, 현재 배치 샘플 수: {len(train_embeddings_batch)}")
                        try:
                             batch_embeddings_array = np.array(train_embeddings_batch)
                             batch_labels_array = np.array(train_labels_batch)

                             # Save batch to a temporary file
                             batch_file_path = os.path.join(temp_embedding_dir, f'{model_name}_train_batch_{len(train_batch_files)}.npz')
                             np.savez(batch_file_path, embeddings=batch_embeddings_array, labels=batch_labels_array)
                             train_batch_files.append(batch_file_path)

                             # Clear batch lists immediately after saving
                             train_embeddings_batch = []
                             train_labels_batch = []
                             original_train_indices = [] # Clear indices too
                             print(f"    훈련 배치 {len(train_batch_files)} 저장 완료. 메모리 해제.")
                             # Optional: Force garbage collection (can sometimes help)
                             gc.collect()


                        except Exception as e:
                            print(f"    오류: 훈련 배치 저장 실패 - 배치 {len(train_batch_files) + 1}, 오류: {e}")
                            # Attempt to clear lists even on error to free memory
                            train_embeddings_batch = []
                            train_labels_batch = []
                            original_train_indices = []


                    if (i + 1) % 50 == 0:
                         print(f"    {i+1}/{len(X_train_paths)} 훈련 파일 처리 완료 (증강 포함).")


            # Process testing data paths in batches
            print("  테스트 데이터 처리 중...")
            test_embeddings_batch = []
            test_labels_batch = []
            # Added index tracking for original samples
            original_test_indices = []
            for i, path in enumerate(X_test_paths):
                original_label_encoded = y_test_encoded[i]
                print(f"    처리 파일 {i+1}/{len(X_test_paths)}: {os.path.basename(path)}")
                # No augmentation for test samples
                 # Ensure noise_audio_paths is passed even if augment_with_noise is False, as the function expects it.
                try:
                     embedding = extract_func(path, model, augment_with_noise=False, noise_audio_paths=noise_audio_paths)

                     if embedding is not None:
                         test_embeddings_batch.append(embedding)
                         test_labels_batch.append(original_label_encoded)
                         original_test_indices.append(i) # Store original index
                     else:
                         print(f"      경고: 파일 '{os.path.basename(path)}' 임베딩 추출 실패 (결과 None).")

                except Exception as e:
                     print(f"    오류: 파일 '{os.path.basename(path)}' 임베딩 추출 중 예외 발생: {e}")
                     import traceback
                     traceback.print_exc(limit=2, file=sys.stdout) # Print limited traceback
                     # Continue to next file even on error


                # Save batch and clear lists if batch size is reached or it's the last sample
                if (len(test_embeddings_batch) >= EMBEDDING_EXTRACTION_BATCH_SIZE) or (i == len(X_test_paths) - 1 and test_embeddings_batch):
                    print(f"    저장 중: 테스트 배치 {len(test_batch_files) + 1}, 현재 배치 샘플 수: {len(test_embeddings_batch)}")
                    try:
                         batch_embeddings_array = np.array(test_embeddings_batch)
                         batch_labels_array = np.array(test_labels_batch)

                         # Save batch to a temporary file
                         batch_file_path = os.path.join(temp_embedding_dir, f'{model_name}_test_batch_{len(test_batch_files)}.npz')
                         np.savez(batch_file_path, embeddings=batch_embeddings_array, labels=batch_labels_array)
                         test_batch_files.append(batch_file_path)

                         # Clear batch lists immediately after saving
                         test_embeddings_batch = []
                         test_labels_batch = []
                         original_test_indices = [] # Clear indices too
                         print(f"    테스트 배치 {len(test_batch_files)} 저장 완료. 메모리 해제.")
                         # Optional: Force garbage collection (can sometimes help)
                         gc.collect()

                    except Exception as e:
                         print(f"    오류: 테스트 배치 저장 실패 - 배치 {len(test_batch_files) + 1}, 오류: {e}")
                         # Attempt to clear lists even on error to free memory
                         test_embeddings_batch = []
                         test_labels_batch = []
                         original_test_indices = []


                if (i + 1) % 20 == 0:
                    print(f"    {i+1}/{len(X_test_paths)} 테스트 파일 처리 완료.")


            # Store the list of batch file paths for this model
            X_train_embeddings[model_name] = train_batch_files
            X_test_embeddings[model_name] = test_batch_files

            # We need the total number of samples and labels for One-Hot encoding and evaluation reports
            # This requires loading all labels again, but not the embeddings themselves
            all_train_labels_filtered = []
            print(f"\n  {model_name} 훈련 배치 파일에서 레이블 수집 중...")
            for batch_file in train_batch_files:
                 try:
                      with np.load(batch_file) as data:
                           all_train_labels_filtered.extend(data['labels'])
                 except Exception as e:
                      print(f"오류: 훈련 배치 파일 로드 실패 (레이블 수집 중) - {batch_file}, 오류: {e}")

            y_train_filtered[model_name] = np.array(all_train_labels_filtered)

            all_test_labels_filtered = []
            print(f"  {model_name} 테스트 배치 파일에서 레이블 수집 중...")
            for batch_file in test_batch_files:
                 try:
                      with np.load(batch_file) as data:
                           all_test_labels_filtered.extend(data['labels'])
                 except Exception as e:
                      print(f"오류: 테스트 배치 파일 로드 실패 (레이블 수집 중) - {batch_file}, 오류: {e}")
            y_test_filtered[model_name] = np.array(all_test_labels_filtered)
            y_test_encoded_filtered[model_name] = y_test_filtered[model_name] # Use filtered test labels for evaluation reports


            print(f"\n  {model_name} 임베딩 추출 및 파일 저장 완료.")
            print(f"  훈련 임베딩 배치 파일 수: {len(X_train_embeddings[model_name])}")
            print(f"  훈련 샘플 수 (총): {y_train_filtered[model_name].shape[0]}")
            print(f"  테스트 임베딩 배치 파일 수: {len(X_test_embeddings[model_name])}")
            print(f"  테스트 샘플 수 (총): {y_test_filtered[model_name].shape[0]}")


            # Check if sufficient samples were extracted for this model (at least 2 samples per class in filtered train data)
            if y_train_filtered[model_name].shape[0] >= 2 and model_name in y_train_filtered and len(np.unique(y_train_filtered[model_name])) >= 2:
                 all_models_extracted_at_least_one_sample = True # At least one model has trainable data


        if all_models_extracted_at_least_one_sample:
             are_embeddings_extracted_successfully = True # Set flag if at least one model has data
        else:
             print("\n경고: 모든 모델에서 훈련 가능한 임베딩 샘플이 부족합니다 (각 클래스 최소 2개 필요).")


    print("\n모델별 임베딩 추출 단계 완료.")

else:
    print("\n데이터 로드, 모델 로드, 또는 모델 정보 누락으로 임베딩 추출을 건너뜁니다.")


# --- Helper function to load batches from saved files ---
def batch_generator(batch_file_paths):
    """Generates batches of embeddings and labels from saved .npz files."""
    # print(f"Debug: batch_generator called with {len(batch_file_paths)} files.") # Debug print
    if not batch_file_paths:
         # print("Debug: batch_file_paths is empty.") # Debug print
         return # Return empty generator

    for file_path in batch_file_paths:
        # print(f"Debug: Loading batch from file: {file_path}") # Debug print
        try:
            with np.load(file_path) as data:
                embeddings = data['embeddings']
                labels = data['labels']
                # Yield batches from the loaded data
                # Since each file is already a batch, we yield the whole file content as one batch
                # print(f"Debug: Yielding batch with shape {embeddings.shape}, {labels.shape}") # Debug print
                yield embeddings, labels
        except Exception as e:
            print(f"오류: 임베딩 배치 파일 로드 실패 - {file_path}, 오류: {e}")
            continue # Skip this file


# --- 4. Prepare Labels for Training (One-Hot Encoding) ---
y_train_one_hot = {}
y_test_one_hot = {}

# Perform one-hot encoding only if embeddings were extracted successfully for at least one model
# and if label_encoder and num_classes are available and valid from data preparation
is_data_ready_for_training = False # Reset and determine based on one-hot encoding success

if are_embeddings_extracted_successfully and 'label_encoder' in locals() and label_encoder is not None and 'num_classes' in locals() and num_classes >= 2:
    print("\n모델별 레이블 One-Hot 인코딩 시작...")
    all_models_encoded_successfully = True # Track if all models that had embeddings are encoded

    # Iterate through models that had successful embedding extraction
    models_with_extracted_data = [name for name, data in y_train_filtered.items() if data.size > 0 and len(np.unique(data)) >= 2]

    if not models_with_extracted_data:
         print("경고: One-Hot 인코딩을 위한 필터링된 데이터가 있는 모델이 없습니다.")
         all_models_encoded_successfully = False


    for model_name in models_with_extracted_data: # Use keys from models with sufficient filtered data
        # Check if filtered labels exist and have at least 2 classes (redundant check, but safe)
        if model_name in y_train_filtered and y_train_filtered[model_name].size > 0 and len(np.unique(y_train_filtered[model_name])) >= 2 and \
           model_name in y_test_filtered and y_test_filtered[model_name].size > 0 and len(np.unique(y_test_filtered[model_name])) >= 2:

             print(f"--- {model_name} 레이블 인코딩 중 ---")
             try:
                 # One-hot Encode the filtered labels using the shared label_encoder
                 y_train_one_hot[model_name] = tf.keras.utils.to_categorical(y_train_filtered[model_name], num_classes=num_classes)
                 y_test_one_hot[model_name] = tf.keras.utils.to_categorical(y_test_filtered[model_name], num_classes=num_classes)
                 print(f"  {model_name} 훈련 레이블 형태 (One-Hot): {y_train_one_hot[model_name].shape}")
                 print(f"  {model_name} 테스트 레이블 형태 (One-Hot): {y_test_one_hot[model_name].shape}")
             except Exception as e:
                 print(f"  오 오류: {model_name} 레이블 One-Hot 인코딩 중 오류 발생: {e}")
                 # Initialize empty arrays if encoding fails
                 y_train_one_hot[model_name] = np.array([])
                 y_test_one_hot[model_name] = np.array([])
                 all_models_encoded_successfully = False # Mark failure


        else:
            print(f"  경고: {model_name}에 대한 필터링된 레이블이 부족하거나 클래스가 2개 미만이어서 One-Hot 인코딩을 건너뜁니다.")
            # Initialize empty arrays if filtered labels are missing or empty or insufficient classes
            y_train_one_hot[model_name] = np.array([])
            y_test_one_hot[model_name] = np.array([])
            all_models_encoded_successfully = False # Mark failure


    if all_models_encoded_successfully and any(arr.size > 0 for arr in y_train_one_hot.values()):
         print("\n모델별 레이블 One-Hot 인코딩 단계 완료.")
         is_data_ready_for_training = True # Set the flag to True if at least one model was encoded successfully
    else:
         print("\n일부 모델의 레이블 One-Hot 인코딩 실패 또는 데이터 부족.")
         is_data_ready_for_training = False # Set the flag to False if any intended model failed encoding


else:
    print("\n임베딩 추출 실패, 클래스 수 부족, 또는 label_encoder 누락으로 레이블 One-Hot 인코딩을 건너뜠습니다.")
    is_data_ready_for_training = False # Set the flag to False


if is_data_ready_for_training:
    print("\n데이터 로드, 임베딩 추출 및 데이터셋 준비 단계 성공.")
else:
    print("\n데이터 로드, 임베딩 추출 및 데이터셋 준비 단계 실패: 훈련에 필요한 데이터가 부족합니다.")


# --- 5. 모델 구축, 학습 및 평가 실행 ---
# Ensure build_classifier_model, train_model, evaluate_and_visualize_model are defined
if 'build_classifier_model' not in locals() or 'train_model' not in locals() or 'evaluate_and_visualize_model' not in locals():
     print("오류: 필요한 모델 구축, 학습 또는 평가 함수가 정의되지 않았습니다. 모델 학습/평가를 건너뜁니다.")
     are_models_trained = False
     trained_models = {}
     training_histories = {}
     evaluation_results = {}

else: # Functions are defined, proceed with training and evaluation if data is ready
    print("\n모델 구축, 학습 및 평가 실행 시작...")

    # Dictionaries to store trained models, histories, and evaluation results
    trained_models = {}
    training_histories = {}
    evaluation_results = {} # To store metrics for comparison

    # Ensure embedding_dims is populated from the extraction step
    if not embedding_dims:
         print("오류: 임베딩 차원 정보가 누락되었습니다 (embedding_dims). 모델 구축/학습을 건너뜁니다.")
         are_models_trained = False
    else: # embedding_dims is defined
        are_models_trained = False # Initialize flag

        if is_data_ready_for_training:
            print("\n모델별 분류기 구축, 학습 및 평가 진행 중...")

            # Iterate through models that have prepared data (using one-hot encoded data availability)
            models_to_process_keys = [name for name, data in y_train_one_hot.items() if data.size > 0]

            if not models_to_process_keys:
                print("경고: 훈련 데이터가 준비된 모델이 없습니다. 학습/평가를 건너뜁니다.")

            for model_name in models_to_process_keys:
                print(f"\n--- {model_name} 모델 처리 중 ---")

                # Check if all necessary data for training and evaluation exists and is not empty for this specific model
                # These checks should align with the validation in build_and_train_classifier and evaluate_and_visualize_model
                if model_name in X_train_embeddings and X_train_embeddings[model_name] and \
                   model_name in y_train_one_hot and y_train_one_hot[model_name].size > 0 and \
                   model_name in X_test_embeddings and X_test_embeddings[model_name] and \
                   model_name in y_test_one_hot and y_test_one_hot[model_name].size > 0 and \
                   model_name in y_test_filtered and y_test_filtered[model_name].size > 0 and \
                   model_name in y_test_encoded_filtered and y_test_encoded_filtered[model_name].size > 0 and \
                   model_name in embedding_dims and 'num_classes' in locals() and num_classes >= 2 and \
                   'label_encoder' in locals() and label_encoder is not None:

                    # Get embedding dimension for the current model
                    embedding_dim = embedding_dims[model_name]
                    print(f"  {model_name} 임베딩 차원: {embedding_dim}")

                    # Build the classifier model
                    classifier_model = build_classifier_model(
                         input_shape=embedding_dim,
                         num_classes=num_classes,
                         learning_rate=0.001 # Define learning rate
                    )

                    # Create batch generators for training and evaluation
                    train_data_generator = batch_generator(X_train_embeddings[model_name])
                    test_data_generator = batch_generator(X_test_embeddings[model_name])

                    # Calculate steps per epoch for training and validation
                    # Sum of samples in all batches / batch size used during training
                    # NOTE: This assumes the batch size used in training (train_model) is the same as EMBEDDING_EXTRACTION_BATCH_SIZE
                    # If not, this calculation needs adjustment.
                    # A simpler approach is to pass the generator directly without steps_per_epoch if the generator handles epochs.
                    # However, Keras fit with generator often requires steps_per_epoch.
                    # Let's assume training batch size is EMBEDDING_EXTRACTION_BATCH_SIZE for now for simplicity.
                    total_train_samples = y_train_filtered[model_name].shape[0]
                    total_test_samples = y_test_filtered[model_name].shape[0]

                    # Ensure steps are at least 1 if there are samples
                    train_steps_per_epoch = max(1, total_train_samples // EMBEDDING_EXTRACTION_BATCH_SIZE)
                    test_steps = max(1, total_test_samples // EMBEDDING_EXTRACTION_BATCH_SIZE) # For evaluation

                    # If total samples are less than batch size, steps_per_epoch will be 0, need to handle this
                    if total_train_samples > 0 and train_steps_per_epoch == 0:
                         train_steps_per_epoch = 1
                    if total_test_samples > 0 and test_steps == 0:
                         test_steps = 1


                    # Train the classifier model using the batch generator
                    print(f"\n  --- {model_name} 모델 학습 (배치 제너레이터 사용) ---")
                    # EarlyStopping 및 ReduceLROnPlateau 콜백 설정 (from train_model function)
                    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
                    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=0.000001)
                    callbacks = [early_stopping, reduce_lr]

                    try:
                         # Use model.fit directly with the generator
                         # Ensure generators are not empty before fitting
                         if total_train_samples > 0 and total_test_samples > 0:
                             history = classifier_model.fit(
                                 train_data_generator,
                                 steps_per_epoch=train_steps_per_epoch,
                                 epochs=50, # Define epochs
                                 validation_data=test_data_generator,
                                 validation_steps=test_steps,
                                 callbacks=callbacks,
                                 verbose=1
                             )
                             print(f"  {model_name} 모델 학습 완료.")
                             trained_models[model_name] = classifier_model
                             training_histories[model_name] = history
                             are_models_trained = True # Set flag to True if at least one model trained

                             # Evaluate the model using the batch generator
                             print(f"\n  --- {model_name} 모델 평가 (배치 제너레이터 사용) ---")
                             # The evaluate_and_visualize_model function currently expects NumPy arrays.
                             # We need to modify evaluate_and_visualize_model or adapt the call here.
                             # Adapting the call here is more consistent with applying changes *here*.

                             # Re-implementing evaluation logic here to use generator
                             print("\n  예측 수행 및 리포트 생성...")
                             try:
                                 # Predict in batches using the generator
                                 # Create a new generator for predict
                                 test_generator_for_predict = batch_generator(X_test_embeddings[model_name])
                                 # Ensure generator is not empty
                                 if total_test_samples > 0:
                                      y_pred_probs = classifier_model.predict(test_generator_for_predict, steps=test_steps)
                                      y_pred = np.argmax(y_pred_probs, axis=1)

                                      # Use the pre-loaded filtered test labels for the report and matrix
                                      y_true_filtered = y_test_encoded_filtered[model_name] # Use the filtered encoded labels

                                      # Ensure the number of predictions matches the number of true labels
                                      if len(y_pred) != len(y_true_filtered):
                                           print(f"경고: {model_name} 예측 결과 수({len(y_pred)})와 실제 레이블 수({len(y_true_filtered)})가 일치하지 않습니다. 평가 리포트/혼동 행렬 생략.")
                                           evaluation_metrics = {'loss': None, 'accuracy': None} # Cannot evaluate reliably
                                      else:
                                           print("\n  분류 리포트:")
                                           # Use target_names from label_encoder if available and matches unique test labels
                                           if label_encoder is not None and len(np.unique(y_true_filtered)) == num_classes:
                                                print(classification_report(y_true_filtered, y_pred, target_names=label_encoder.classes_))
                                           else:
                                                # Fallback without target names
                                                print(classification_report(y_true_filtered, y_pred))
                                                print("  경고: 레이블 인코더 또는 클래스 불일치로 인해 target_names를 사용할 수 없습니다.")


                                           # --- 혼동 행렬 시각화 ---
                                           print("\n  혼동 행렬 시각화:")
                                           # Ensure the unique classes in the filtered test data match the number of classes for the matrix size
                                           unique_test_labels_filtered = np.unique(y_true_filtered)
                                           if len(unique_test_labels_filtered) == num_classes:
                                                # Ensure labels for confusion matrix calculation cover all unique predicted and true labels
                                                all_possible_labels = np.unique(np.concatenate((y_true_filtered, y_pred)))
                                                # If label_encoder is available, use its classes order for consistent matrix
                                                if label_encoder is not None and hasattr(label_encoder, 'classes_'):
                                                    labels_for_matrix = np.arange(len(label_encoder.classes_))
                                                    # Filter labels_for_matrix to only include those present in y_true_filtered or y_pred for smaller datasets
                                                    present_labels_in_data = np.unique(np.concatenate((y_true_filtered, y_pred))).tolist()
                                                    labels_for_matrix = [l for l in labels_for_matrix if l in present_labels_in_data]

                                                    cm = confusion_matrix(y_true_filtered, y_pred, labels=labels_for_matrix)

                                                    plt.figure(figsize=(max(6, len(labels_for_matrix)), max(5, len(labels_for_matrix)))) # Adjust figure size
                                                    # Use label_encoder classes for tick labels if available, filtered to present labels
                                                    tick_labels = [label_encoder.classes_[i] for i in labels_for_matrix]
                                                    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                                                               xticklabels=tick_labels, yticklabels=tick_labels)
                                                else:
                                                     # Fallback without labels if label_encoder or classes don't match
                                                     # Use unique labels from the data for matrix labels if label_encoder is not fully usable
                                                     labels_for_matrix = np.unique(np.concatenate((y_true_filtered, y_pred)))
                                                     cm = confusion_matrix(y_true_filtered, y_pred, labels=labels_for_matrix)
                                                     plt.figure(figsize=(max(6, len(labels_for_matrix)), max(5, len(labels_for_matrix)))) # Adjust figure size
                                                     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues') # Fallback without labels
                                                plt.xlabel('예측 레이블')
                                                plt.ylabel('실제 레이블')
                                                plt.title(f'{model_name} 혼동 행렬')
                                                plt.show()
                                           else:
                                                print("  경고: 필터링된 테스트 데이터에 모든 클래스가 포함되지 않아 혼동 행렬을 생성할 수 없습니다.")

                                           # Evaluate model to get loss and accuracy
                                           loss, accuracy = classifier_model.evaluate(test_data_generator, steps=test_steps, verbose=0)
                                           evaluation_metrics = {'loss': loss, 'accuracy': accuracy}
                                           print(f"  테스트 세트 손실: {loss:.4f}")
                                           print(f"  테스트 세트 정확도: {accuracy:.4f}")
                                 else:
                                      print("경고: 테스트 샘플이 부족하여 예측/평가를 건너뜁니다.")
                                      evaluation_metrics = {'loss': None, 'accuracy': None}


                             except Exception as e:
                                 print(f"  오류: {model_name} 모델 예측 또는 리포트 생성 중 오류 발생: {e}")
                                 evaluation_metrics = {'loss': None, 'accuracy': None} # Mark evaluation as failed


                             evaluation_results[model_name] = evaluation_metrics # Store evaluation metrics

                             # --- 학습 과정 시각화 ---
                             print("\n  학습 과정 시각화:")
                             if history is not None and hasattr(history, 'history'):
                                 plt.figure(figsize=(12, 5))

                                 # Accuracy plot
                                 plt.subplot(1, 2, 1)
                                 plt.plot(history.history.get('accuracy', []), label='훈련 정확도')
                                 if 'val_accuracy' in history.history:
                                      plt.plot(history.history['val_accuracy'], label='검증 정확도')
                                 plt.xlabel('에폭')
                                 plt.ylabel('정확도')
                                 plt.title(f'{model_name} 훈련 및 검증 정확도')
                                 plt.legend()

                                 # Loss plot
                                 plt.subplot(1, 2, 2)
                                 plt.plot(history.history.get('loss', []), label='훈련 손실')
                                 if 'val_loss' in history.history:
                                      plt.plot(history.history['val_loss'], label='검증 손실')
                                 plt.xlabel('에폭')
                                 plt.ylabel('손실')
                                 plt.title(f'{model_name} 훈련 및 검증 손실')
                                 plt.legend()

                                 plt.show()
                             else:
                                 print("  경고: 학습 기록(history)이 없어 그래프를 그릴 수 없습니다.")

                         else: # total_train_samples <= 0 or total_test_samples <= 0
                              print("경고: 훈련 또는 테스트 샘플이 부족하여 모델 학습/평가를 건너뜁니다.")
                              trained_models[model_name] = None
                              training_histories[model_name] = None
                              evaluation_results[model_name] = {'loss': None, 'accuracy': None} # Store None for metrics if skipped


                    except Exception as e:
                         print(f"  오류: {model_name} 모델 학습 중 오류 발생: {e}")
                         trained_models[model_name] = None
                         training_histories[model_name] = None
                         evaluation_results[model_name] = {'loss': None, 'accuracy': None} # Store None for metrics if skipped


                else:
                    print(f"경고: {model_name} 모델 학습/평가를 위한 데이터 또는 필수 변수가 부족합니다. 건너뜁니다.")
                    trained_models[model_name] = None
                    training_histories[model_name] = None
                    evaluation_results[model_name] = {'loss': None, 'accuracy': None} # Store None for metrics if skipped


            print("\n모델 구축, 학습 및 평가 실행 단계 완료.")

        else:
            print("\n데이터 준비 실패로 모델 학습 및 평가를 건너뜁니다.")
            are_models_trained = False # Ensure flag is False if data not ready


# --- Cleanup temporary embedding directory ---
if temp_embedding_dir and os.path.exists(temp_embedding_dir):
    print(f"\n임시 임베딩 디렉토리 삭제 중: {temp_embedding_dir}")
    try:
        # Check if directory is empty before attempting removal (optional)
        # if not os.listdir(temp_embedding_dir):
        #      print("Debug: 임시 디렉토리가 비어 있습니다.")
        shutil.rmtree(temp_embedding_dir)
        print("임시 디렉토리 삭제 완료.")
    except Exception as e:
        print(f"오류: 임시 디렉토리 삭제 실패: {e}")


# --- 6. 모델 성능 비교 (요약) ---
print("\n6. 모델 성능 비교 (요약) 중...")
print("\n--- 모델별 최종 성능 비교 ---")
if evaluation_results and any(metrics.get('accuracy') is not None for metrics in evaluation_results.values()):
    # Sort results by accuracy (optional, but helps in comparison)
    sorted_results = sorted(evaluation_results.items(), key=lambda item: item[1].get('accuracy') if item[1].get('accuracy') is not None else -1, reverse=True)

    for model_name, metrics in sorted_results:
        print(f"  {model_name}:")
        print(f"    테스트 손실: {metrics.get('loss'):.4f}" if metrics.get('loss') is not None else "    테스트 손실: N/A")
        print(f"    테스트 정확도: {metrics.get('accuracy'):.4f}" if metrics.get('accuracy') is not None else "    테스트 정확도: N/A")
else:
    print("평가 결과가 없어 모델 성능 비교를 수행할 수 없습니다.")


# --- 7. 새로운 오디오 파일에 대한 예측 (예시) ---
print("\n7. 새로운 오디오 파일 예측 예시 중...")
print("\n--- 새로운 오디오 파일 예측 예시 ---")

# Choose one of the trained models for prediction, e.g., the best performing one or YAMNet
# If trained_models and any(model is not None for model in trained_models.values()): # Keep this check
if any(model is not None for model in trained_models.values()): # Simplified check if any model was trained
    # Choose the first successfully trained model for prediction example
    model_to_predict_name = None
    for name, model in trained_models.items():
         if model is not None:
              model_to_predict_name = name
              break
    model_to_predict = trained_models.get(model_to_predict_name) # Get the model object


    if model_to_predict is not None:
        print(f"\n예측에 사용할 모델: {model_to_predict_name}")

        # Need a sample audio file path for prediction
        # Use the first path from the original combined list if available, or a dummy file
        predict_audio_path = None

        # Attempt to use the first path from the original combined list if it was populated
        if 'all_audio_paths' in locals() and all_audio_paths:
            predict_audio_path = all_audio_paths[0]
            print(f"예측을 위해 데이터셋에서 샘플 파일 선택: {predict_audio_path}")
        elif 'DEEPSHIP_BASE_PATH' in locals() and os.path.exists(os.path.join(DEEPSHIP_BASE_PATH, 'Cargo', '103.wav')): # Fallback to a known DeepShip file
            predict_audio_path = os.path.join(DEEPSHIP_BASE_PATH, 'Cargo', '103.wav')
            print(f"예측을 위해 DeepShip에서 샘플 파일 선택: {predict_audio_path}")
        # Add other fallbacks here if needed

        # Ensure the chosen model, label_encoder, and yamnet_model are available for prediction
        # Need the original YAMNet model for preprocessing in predict_on_new_audio
        if predict_audio_path and os.path.exists(predict_audio_path) and \
           model_to_predict is not None and \
           'label_encoder' in locals() and label_encoder is not None and \
           'loaded_models' in locals() and loaded_models.get('YAMNet') is not None:

            # Ensure predict_on_new_audio is defined and available
            if 'predict_on_new_audio' not in locals():
                 print("오류: 'predict_on_new_audio' 함수가 정의되지 않았습니다. 예측을 건너뜁니다.")
            else:
                # Call the predict_on_new_audio function
                # Pass the original YAMNet model for preprocessing
                predicted_label = predict_on_new_audio(model_to_predict, loaded_models.get('YAMNet'), label_encoder, predict_audio_path)
                # The predicted label is printed inside the function

        else:
            print("\n예측을 수행할 수 없습니다:")
            if not predict_audio_path or not os.path.exists(predict_audio_path):
                 print("  예측할 오디오 파일을 찾을 수 없습니다.")
            if model_to_predict is None:
                 print(f"  학습된 '{model_to_predict_name}' 모델이 없습니다.")
            if 'label_encoder' not in locals() or label_encoder is None:
                 print("  LabelEncoder가 없습니다.")
            if 'loaded_models' not in locals() or loaded_models.get('YAMNet') is not None:
                 print("  YAMNet 모델이 없습니다 (예측 전처리용).")

else: # No models were trained
    print("\n학습된 모델이 없어 예측을 수행할 수 없습니다.")


print("\n모델별 전체 파이프라인 실행 및 결과 비교 단계 완료.")

# Subtask is completed.

## Summary:

### Data Analysis Key Findings

*   The refactoring process successfully defined functions for initial setup, data loading/preparation, audio preprocessing/embedding extraction, model loading, classifier building/training, evaluation/visualization, and prediction.
*   The data loading and preparation step (`load_and_prepare_dataset`) was executed but failed to collect any audio files from either the cloned DeepShip directory or the specified MBARI noise data directory.
*   This lack of data (0 samples collected, 0 classes identified) prevented the subsequent steps of embedding extraction, label one-hot encoding, model training, and evaluation from executing, as designed by the implemented data availability checks.
*   The pipeline correctly identified the data insufficiency and skipped the computationally intensive steps.

### Insights or Next Steps

*   **Critical Data Acquisition:** The most critical next step is to acquire the necessary audio data. This includes:
    *   Verifying the structure of the cloned DeepShip repository (`/content/DeepShip`) to understand why the `load_and_prepare_dataset` function is not finding the `.wav` files within the expected class subdirectories. Manual inspection of the `/content/DeepShip` directory contents might be necessary.
    *   Acquiring a substantial amount of MBARI noise data (or other relevant noise data) and placing it in the designated `MBARI_NOISE_BASE_DIR` (`/content/MBARI_noise_data`). The previous attempts to download sample files were not sufficient or successful in populating this directory with enough noise data.
*   **Re-run Pipeline:** Once sufficient data for both 'ship' and 'noise' classes (at least 2 samples per class for stratified splitting) is placed in the correct directories, the entire pipeline (starting from Step 6 which executes data loading, embedding, training, evaluation) needs to be re-executed. The existing code is designed to handle the process once data is available.
*   **Review Data Loading Logic:** If DeepShip files are still not found after verifying the directory structure, the `load_and_prepare_dataset` function's DeepShip traversal logic may need further debugging or adjustment based on the actual file paths.